In [ ]:
import json
import pandas as pd
import numpy as np
import datasets
import transformers
import torch
import torch.nn as nn
import torch.utils.data
import warnings
from torch.utils.data import DataLoader

warnings.filterwarnings('ignore')

# following function is adopted from bert4keras package (https://github.com/bojone/bert4keras)
# we do not import this package to avoid compadibility issues (keras < 2.3.1 is required for this package, while a later version is already used)
# if bert4keras package is already installed, this function can be loaded as follows:
# from bert4keras.snippets import text_segmentate

def text_segmentate(text, maxlen, seps='\n', strips=None):
    """将文本按照标点符号划分为若干个短句
    """
    text = text.strip().strip(strips)
    if seps and len(text) > maxlen:
        pieces = text.split(seps[0])
        text, texts = '', []
        for i, p in enumerate(pieces):
            if text and p and len(text) + len(p) > maxlen - 1:
                texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
                text = ''
            if i + 1 == len(pieces):
                text = text + p
            else:
                text = text + p + seps[0]
        if text:
            texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
        return texts
    else:
        return [text]


# following function is adopted from https://github.com/Pzeyang/task-for-authorship-verification
# a custom version tailored to our project will be added later

def get_data(jsonl_dataset_path):
    """
    Get data from JSONL dataset. Used in plain_pipeline and pipeline
    """

    with open(jsonl_dataset_path, 'r') as f:

        datas = []
        for l in f:
            data = json.loads(l)
            text1 = text_segmentate(data['pair'][0], maxlen=510, seps='.?!')
            text2 = text_segmentate(data['pair'][1], maxlen=510, seps='.?!')
            while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
            datas.append((text1, text2, str(data['id'])))

        return datas

# different data extractors for different types of input. See description to find in which pipeline each one should be used

def get_data_from_two_textfiles(text1_path, text2_path):
    """
    Get data from a two text files, one for each fragment. Used in pipeline 
    """

    print("Getting data from raw texts")

    datas = []
    with open(text1_path, 'r') as text1, open(text2_path, 'r') as text2:
        text1, text2 = text1.read(), text2.read()
        text1 = text_segmentate(text1, maxlen=510, seps='.?!')
        text2 = text_segmentate(text2, maxlen=510, seps='.?!')
        while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
        datas.append((text1, text2))

    return datas

def get_data_from_single_textfile(text_path):
    """
    Get data from a text file that contains two texts and a separator. Currently not used
    """

    print("Getting data from single raw text file")

    datas = []
    with open(text_path, 'r') as text:
        text = text.read()
        text1, text2 = text.split("$&*&*&$")
        text1 = text_segmentate(text1, maxlen=510, seps='.?!')
        text2 = text_segmentate(text2, maxlen=510, seps='.?!')
        while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
        datas.append((text1, text2))

    return datas

def get_data_from_combined_texts(text_or_list):
    """
    Get data from raw text that contains two fragments and a separater, or from a list of texts,
    each of them containing two fragments and a separater. Used in pipeline_onetext. The ONLY type
    of data processor for LIME inputs
    """

    print("Getting data from raw text")

    datas = []

    print(type(text_or_list), len(text_or_list))
    if not isinstance(text_or_list, str):
        for text_variant in text_or_list:
            text1, text2 = text_variant.split("$&*&*&$")
            text1 = text_segmentate(text1, maxlen=510, seps='.?!')
            text2 = text_segmentate(text2, maxlen=510, seps='.?!')
            while len(text1) < 30 or len(text2) < 30:
                    if len(text1) < 30:
                        n_text1 = []
                        for i in range(30):
                            for sent in text1:
                                n_text1.append(sent)
                        text1 = n_text1
                    elif len(text2) < 30:
                        n_text2 = []
                        for i in range(30):
                            for sent in text2:
                                n_text2.append(sent)
                        text2 = n_text2
            datas.append((text1, text2))
    else:
        text1, text2 = text_or_list.split("$&*&*&$")
        text1 = text_segmentate(text1, maxlen=510, seps='.?!')
        text2 = text_segmentate(text2, maxlen=510, seps='.?!')
        while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
        datas.append((text1, text2))
    return datas


global tokenizer 
tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
def tokenize_function(example):
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    return tokenizer(example['0'][:30], example['1'][:30], truncation=True, padding='max_length', max_length=255)

# used to creat text input for pipelines and explainers
def combine_texts(index, write=False):
    """
    Combine a pair of texts from dataset with a separator and turn into a single text
    """

    text1 = orig_data['pair'][index][0]
    text2 = orig_data['pair'][index][1]
    text_combined = text1 + "$&*&*&$" + text2
    
    if write:
        name = "textcomb{}.txt".format(index)
        with open(name, 'w') as textcomb:
            textcomb.write(text_combined)

    return(text_combined)

## Initialize a final classifier (identical to FinalNetAvg in Final_model_PT)

In [ ]:
class FinalNetAvg(nn.Module):

    def __init__(self, num_classes=2):
        super(FinalNetAvg, self).__init__()
        self.avgpool = nn.AdaptiveAvgPool2d((1, 768))
        self.classifier = nn.Sequential(
            nn.Linear(768, 16),
            nn.ReLU(),
            nn.Linear(16, num_classes)
        )

    def forward(self, x):
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

## Get a standard JSONL dataset with 100 pairs and get corresponding truth labels

Create a standard JSONL with a set number of pairs

In [ ]:
df = pd.read_json("..\BertAA_content\Data\pan20-authorship-verification-training-small.jsonl", lines = True)
df = df.sample(n = 1)
df.to_json("..\BertAA_content\Data\pan20-authorship-verification-training-small-one.jsonl", orient='records', lines = True)

In [ ]:
# load untokenized evaluation set 
from datasets import load_from_disk
df = load_from_disk("..\\BertAA_content\\Data\\100_examples")
df = df.to_pandas()

In [ ]:
orig_data = pd.read_json("..\BertAA_content\Data\pan20-authorship-verification-training-small-verysmall.jsonl", lines = True)
all_trues = pd.read_json("..\BertAA_content\Data\pan20-authorship-verification-training-small-truth.jsonl", lines = True)

In [ ]:
trues = pd.merge(orig_data, all_trues, on=['id'], how='inner')
trues['same'] = trues['same'].astype(int)
labels = trues['same'].array

#np.equal(predictions, labels)

In [ ]:
trues.to_csv("100 examples to explain.csv", index=False)

## Get input data in custom format (otherwise use combine_text function)

In [ ]:
"""
Create a pair of texts from dataset
"""

with open("text3.txt", 'w') as text1, open("text4.txt", 'w') as text2:
    text1.write(orig_data['pair'][0][0])
    text2.write(orig_data['pair'][0][1])

In [ ]:
"""
Combine a pair of texts from files with a separator and turn into a single text
"""

with open("text3.txt", 'r') as text1, open("text4.txt", 'r') as text2, open("textcomb2.txt", 'w') as textcomb:
    text_combined = text1.read() + "$&*&*&$" + text2.read()
    textcomb.write(text_combined)


In [ ]:
def combine_segments_from_pd(textindex, segmentindex, sep_option=0, write=False):
    """
    Combine a pair of segments from pandas dataset with a separator and turn into a single text
    """

    text1 = df['0'][textindex][segmentindex]
    text2 = df['1'][textindex][segmentindex]
    sep = "$&*&*&$" if sep_option == 0 else "[SEP]"
    text_combined = text1 + sep + text2
    
    if write:
        name = "textcomb{}_{}.txt".format(textindex, segmentindex)
        with open(name, 'w') as textcomb:
            textcomb.write(text_combined)

    return(text_combined)

In [ ]:
segm00 = combine_segments_from_pd(0,0,1)
segm00

# Pipelines

In [ ]:
data_path = "..\BertAA_content\Data\pan20-authorship-verification-training-small-one.jsonl"

def plain_pipeline(data_path):
    """
    Pipeline for input from a regular JSONL dataset 
    """

    segmented_data = get_data(data_path)
    dataset = datasets.Dataset.from_pandas(pd.DataFrame(segmented_data))
    del segmented_data

    print("Tokenization...")

    #only ititialize tokenizer if you don't do it before calling the function (which is faster)
    #global tokenizer 
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    tokenized_dataset = dataset.map(tokenize_function)
    tokenized_dataset = tokenized_dataset.remove_columns(['0', '1', '2'])
    #print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))

    flat_dataset = tokenized_dataset.to_pandas()
    flat_dataset = flat_dataset.explode(['input_ids', 'token_type_ids', "attention_mask"]).reset_index(drop=True)
    dataset = datasets.Dataset.from_pandas(flat_dataset)

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results2\checkpoint-225000")
    model_feature_extract.to(device)
    print("Obtaining embeddings...")

    dataset.set_format('torch')
    eval_dataloader = DataLoader(dataset, shuffle=False, batch_size=30)

    eval_outputs = torch.Tensor()
    eval_outputs = eval_outputs.to(device)

    with torch.no_grad():
        for i, batch in enumerate(eval_dataloader):
            if i % 10 == 0:
                print(">{} processing batch {}/{}".format(i//10*">", i, len(eval_dataloader))) 

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model_feature_extract(**batch, output_hidden_states=True)
            cls = outputs.hidden_states[-1][:,0,:] # obtain last hidden layer's CLS tokens. [:,0,:] meaning: ':' for all sequences, '0' for first token in sequence, ':' for all 768 hidden layers
            eval_outputs = torch.cat((eval_outputs, cls), 0)

    eval_outputs = torch.reshape(eval_outputs, (len(eval_dataloader), 30, 768))

    print("Making predictions...")

    model_classify = FinalNetAvg()
    model_classify.load_state_dict(torch.load(r"..\BertAA_content\Model\Classifier\model.pth"))
    model_classify.to(device)

    logits = model_classify(eval_outputs)
    predictions = torch.argmax(logits, dim=-1)
    print("Done!")
    return predictions.cpu().numpy()

## Pipeline with input either from regular JSONL dataset (1 argument) or from a pair of texts (2 arguments)

In [ ]:
data_path = "..\BertAA_content\Data\pan20-authorship-verification-training-small-one.jsonl"

def pipeline(data_path, data_path2=None, mode='probs'):
    """
    Pipeline with input either from regular JSONL dataset (1 argument) or from a pair of texts (2 arguments)
    """

    segmented_data = get_data_from_two_textfiles(data_path, data_path2) if data_path2 else get_data(data_path)
    dataset = datasets.Dataset.from_pandas(pd.DataFrame(segmented_data))
    del segmented_data

    print("Tokenization...")

    #only ititialize tokenizer if you don't do it before calling the function (which is faster)
    #global tokenizer 
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    tokenized_dataset = dataset.map(tokenize_function)
    #print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))
    
    flat_dataset = tokenized_dataset.to_pandas()
    flat_dataset = flat_dataset.drop(['0', '1'], axis=1)
    if '2' in flat_dataset: #we may or may not have this column depending on the input type
         flat_dataset = flat_dataset.drop(['2'], axis=1)
    flat_dataset = flat_dataset.explode(['input_ids', 'token_type_ids', "attention_mask"]).reset_index(drop=True)
    dataset = datasets.Dataset.from_pandas(flat_dataset)

    global datacheck
    datacheck = flat_dataset

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results2\checkpoint-225000")
    model_feature_extract.to(device)
    print("Obtaining embeddings...")

    dataset.set_format('torch')
    eval_dataloader = DataLoader(dataset, shuffle=False, batch_size=30)

    eval_outputs = torch.Tensor()
    eval_outputs = eval_outputs.to(device)

    model_feature_extract.eval()
    with torch.no_grad():
        for i, batch in enumerate(eval_dataloader):
            if i % 10 == 0:
                print(">{} processing batch {}/{}".format(i//10*">", i, len(eval_dataloader))) 

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model_feature_extract(**batch, output_hidden_states=True)
            cls = outputs.hidden_states[-1][:,0,:] # obtain last hidden layer's CLS tokens. [:,0,:] meaning: ':' for all sequences, '0' for first token in sequence, ':' for all 768 hidden layers
            eval_outputs = torch.cat((eval_outputs, cls), 0)

    eval_outputs = torch.reshape(eval_outputs, (len(eval_dataloader), 30, 768))

    print("Making predictions...")

    model_classify = FinalNetAvg()
    model_classify.load_state_dict(torch.load(r"..\BertAA_content\Model\Classifier\model.pth"))
    model_classify.to(device)

    model_classify.eval()
    with torch.no_grad():
        logits = model_classify(eval_outputs)
    
    predictions = []
    for prediction in logits:
        if mode == 'labels':
            prediction = torch.argmax(prediction, dim=-1)
            prediction = prediction.cpu().numpy()
        elif mode == 'probs':
            m = nn.Softmax()
            prediction = m(prediction)
            prediction = prediction.cpu().numpy()
            prediction = np.around(prediction, decimals=3)
            #prediction = prediction.tolist()
        else:
            prediction = prediction.cpu().numpy()
        predictions.append(prediction)
    print("Done!")
    return predictions

In [ ]:
predictions_from_pair = pipeline("text1.txt", "text2.txt", mode='probs')

In [ ]:
predictions_from_jsonl = pipeline("..\BertAA_content\Data\pan20-authorship-verification-training-small-one.jsonl", mode='probs')

## Pipeline with input from a combined text or a list of combined texts

In [ ]:
def pipeline_onetext(data_path, mode='probs'):
    """
    Pipeline with input from a combined text or a list of combined texts
    """

    segmented_data = get_data_from_combined_texts(data_path)
    dataset = datasets.Dataset.from_pandas(pd.DataFrame(segmented_data))
    del segmented_data

    print("Tokenization...")

    #only ititialize tokenizer if you don't do it before calling the function (which is faster)
    #global tokenizer 
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    tokenized_dataset = dataset.map(tokenize_function)
    #print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))

    flat_dataset = tokenized_dataset.to_pandas()
    flat_dataset = flat_dataset.drop(['0', '1'], axis=1)
    if '2' in flat_dataset: #we may or may not have this column depending on the input type
         flat_dataset = flat_dataset.drop(['2'], axis=1)
    flat_dataset = flat_dataset.explode(['input_ids', 'token_type_ids', "attention_mask"]).reset_index(drop=True)
    dataset = datasets.Dataset.from_pandas(flat_dataset)

    global datacheck
    datacheck = flat_dataset

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
    model_feature_extract.to(device)
    print("Obtaining embeddings...")

    dataset.set_format('torch')
    eval_dataloader = DataLoader(dataset, shuffle=False, batch_size=30)

    eval_outputs = torch.Tensor()
    eval_outputs = eval_outputs.to(device)

    model_feature_extract.eval()
    with torch.no_grad():
        for i, batch in enumerate(eval_dataloader):
            print(batch)
            step = 10 if (len(eval_dataloader) < 100) else 100
            if i % step == 0:
                print(">{} processing item {}/{}".format(int((i/len(eval_dataloader))*10)*">", i, len(eval_dataloader))) 

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model_feature_extract(**batch, output_hidden_states=True)
            cls = outputs.hidden_states[-1][:,0,:] # obtain last hidden layer's CLS tokens. [:,0,:] meaning: ':' for all sequences, '0' for first token in sequence, ':' for all 768 hidden layers
            eval_outputs = torch.cat((eval_outputs, cls), 0)

    eval_outputs = torch.reshape(eval_outputs, (len(eval_dataloader), 30, 768))

    #Save the text embedding for future analysis
    global embedding
    embedding = eval_outputs

    print("Making predictions...")

    model_classify = FinalNetAvg()
    model_classify.load_state_dict(torch.load(r"..\BertAA_content\Model\Classifier\model.pth"))
    model_classify.to(device)

    model_classify.eval()
    with torch.no_grad():
        logits = model_classify(eval_outputs)
    
    predictions = []
    for prediction in logits:
        if mode == 'labels':
            prediction = torch.argmax(prediction, dim=-1)
            prediction = prediction.cpu().numpy()
        elif mode == 'probs':
            m = nn.Softmax()
            prediction = m(prediction)
            prediction = prediction.cpu().numpy()
            prediction = np.around(prediction, decimals=3)
            #prediction = prediction.tolist()
        else:
            prediction = prediction.cpu().numpy()
        predictions.append(prediction)
    print("Done!")
    return np.array(predictions)

In [ ]:
def get_data_from_combined_segments(text_or_list):
    """
    Get data from raw text that contains two fragments and a separater, or from a list of texts,
    each of them containing two fragments and a separater. Used in pipeline_onetext. The ONLY type
    of data processor for LIME inputs
    """

    #print("Getting data from raw text")

    datas = []

    #print(type(text_or_list), len(text_or_list))
    if not isinstance(text_or_list, str):
        for text_variant in text_or_list:
            #print(text_variant)
            text1, text2 = text_variant.split("$&*&*&$")
            datas.append(([text1], [text2]))
    else:
        text1, text2 = text_or_list.split("$&*&*&$")
        datas.append(([text1], [text2]))
    return datas

def pipeline_onesegment(data_path, mode='probs'):
    
    segmented_data = get_data_from_combined_segments(data_path)
    dataset = datasets.Dataset.from_pandas(pd.DataFrame(segmented_data))
    del segmented_data

    #print("Tokenization...")

    #only ititialize tokenizer if you don't do it before calling the function (which is faster)
    #global tokenizer 
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    tokenized_dataset = dataset.map(tokenize_function)
    #print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))

    flat_dataset = tokenized_dataset.to_pandas()
    flat_dataset = flat_dataset.drop(['0', '1'], axis=1)
    if '2' in flat_dataset: #we may or may not have this column depending on the input type
         flat_dataset = flat_dataset.drop(['2'], axis=1)
    flat_dataset = flat_dataset.explode(['input_ids', 'token_type_ids', "attention_mask"]).reset_index(drop=True)
    dataset = datasets.Dataset.from_pandas(flat_dataset)

    global datacheck
    datacheck = flat_dataset

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
    #model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-180000")
    model_feature_extract.to(device)
    #print("Obtaining embeddings...")

    dataset.set_format('torch')
    global eval_dataloader
    eval_dataloader = DataLoader(dataset, shuffle=False, batch_size=30)

    eval_outputs = torch.Tensor()
    eval_outputs = eval_outputs.to(device)

    model_feature_extract.eval()
    with torch.no_grad():
        for i, batch in enumerate(eval_dataloader):
            #print(batch)
            #step = 10 if (len(eval_dataloader) < 100) else 100
            #if i % step == 0:
            #    print(">{} processing item {}/{}".format(int((i/len(eval_dataloader))*10)*">", i, len(eval_dataloader))) 

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model_feature_extract(**batch, output_hidden_states=True)
            cur_logits = outputs.logits
            eval_outputs = torch.cat((eval_outputs, cur_logits), 0)

    #eval_outputs = torch.reshape(eval_outputs, (len(eval_dataloader), 30, 768))

    #Save the text embedding for future analysis
    #global logits
    logits = eval_outputs

    predictions = []
    for prediction in logits:
        if mode == 'labels':
            prediction = torch.argmax(prediction, dim=-1)
            prediction = prediction.cpu().numpy()
        elif mode == 'probs':
            m = nn.Softmax()
            prediction = m(prediction)
            prediction = prediction.cpu().numpy()
            prediction = np.around(prediction, decimals=3)
            #prediction = prediction.tolist()
        else:
            prediction = prediction.cpu().numpy()
        predictions.append(prediction)
    #print("Done!")
    return np.array(predictions)

def get_data_from_listed_segments(text_or_list):
    """
    Get data from raw text that contains two fragments and a separater, or from a list of texts,
    each of them containing two fragments and a separater. Used in pipeline_onetext. The ONLY type
    of data processor for LIME inputs
    """

    print("Getting data from raw text")

    datas = []

    print(type(text_or_list), len(text_or_list))
    if not isinstance(text_or_list[0], str):
        for text_variant in text_or_list:
            #print(text_variant)
            text1, text2 = text_variant[0], text_variant[1]
            datas.append(([text1], [text2]))
    else:
        text1, text2 = text_or_list[0], text_or_list[1]
        datas.append(([text1], [text2]))
    return datas

def pipeline_twosegments(data_path, mode='probs'):

    print(data_path)
    
    segmented_data = get_data_from_listed_segments(data_path)
    dataset = datasets.Dataset.from_pandas(pd.DataFrame(segmented_data))
    del segmented_data

    print("Tokenization...")

    #only ititialize tokenizer if you don't do it before calling the function (which is faster)
    #global tokenizer 
    #tokenizer = transformers.AutoTokenizer.from_pretrained("bert-base-cased")
    tokenized_dataset = dataset.map(tokenize_function)
    #print(tokenizer.decode(tokenized_dataset[0]['input_ids'][0]))

    flat_dataset = tokenized_dataset.to_pandas()
    flat_dataset = flat_dataset.drop(['0', '1'], axis=1)
    if '2' in flat_dataset: #we may or may not have this column depending on the input type
         flat_dataset = flat_dataset.drop(['2'], axis=1)
    flat_dataset = flat_dataset.explode(['input_ids', 'token_type_ids', "attention_mask"]).reset_index(drop=True)
    dataset = datasets.Dataset.from_pandas(flat_dataset)

    global datacheck
    datacheck = flat_dataset

    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    torch.cuda.empty_cache()

    model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")
    #model_feature_extract = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-180000")
    model_feature_extract.to(device)
    print("Obtaining embeddings...")

    dataset.set_format('torch')
    global eval_dataloader
    eval_dataloader = DataLoader(dataset, shuffle=False, batch_size=30)

    eval_outputs = torch.Tensor()
    eval_outputs = eval_outputs.to(device)

    model_feature_extract.eval()
    with torch.no_grad():
        for i, batch in enumerate(eval_dataloader):
            #print(batch)
            step = 10 if (len(eval_dataloader) < 100) else 100
            if i % step == 0:
                print(">{} processing item {}/{}".format(int((i/len(eval_dataloader))*10)*">", i, len(eval_dataloader))) 

            batch = {k: v.to(device) for k, v in batch.items()}
            outputs = model_feature_extract(**batch, output_hidden_states=True)
            cur_logits = outputs.logits
            eval_outputs = torch.cat((eval_outputs, cur_logits), 0)

    #eval_outputs = torch.reshape(eval_outputs, (len(eval_dataloader), 30, 768))

    #Save the text embedding for future analysis
    global logits
    logits = eval_outputs

    predictions = []
    for prediction in logits:
        if mode == 'labels':
            prediction = torch.argmax(prediction, dim=-1)
            prediction = prediction.cpu().numpy()
        elif mode == 'probs':
            m = nn.Softmax()
            prediction = m(prediction)
            prediction = prediction.cpu().numpy()
            prediction = np.around(prediction, decimals=3)
            #prediction = prediction.tolist()
        else:
            prediction = prediction.cpu().numpy()
        predictions.append(prediction)
    print("Done!")
    return np.array(predictions)

In [ ]:
res = pipeline_twosegments([segments[0][0][0], segments[0][1][0]], mode='logits')

In [ ]:
res = pipeline_onesegment(segm00.replace("[SEP]", "$&*&*&$"), mode='logits')
res[0]

In [ ]:
np.subtract(res[0], res[0])

# Experiments with attention

In [ ]:
from bertviz import head_view, model_view
import transformers
import torch

In [ ]:
respd = pd.read_csv("res_100.csv")

In [ ]:
segm00 = """She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, 
she reached down and pulled out the spare key Joey kept there for her. She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs into his room. 
She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house. "MARY!" Joey yelled, standing dumb-struck in the doorway.[SEP]"Joh..n." 
her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. They jerked apart and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. 
Then she bit out,"Nice going laser. As usual I"ll have to fix your mess." Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it."""

In [ ]:
model1 = transformers.AutoModelForSequenceClassification.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000", output_attentions=True).eval()
tokenizer1 = transformers.AutoTokenizer.from_pretrained(r"..\BertAA_content\Model\Checkpoints\results_45000\checkpoint-225000")

model2 = transformers.AutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels = 2, output_attentions=True)
tokenizer2 = transformers.AutoTokenizer.from_pretrained("bert-base-cased")

In [ ]:
def get_head_view(segm, model, tokenizer, attention_layer):
    model = model.eval()
    text1, text2 = segm.split("$&*&*&$")
    inputs = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

    #sentence_a = "The cat sat on the mat"
    #sentence_b = "The cat lay on the rug"
    #inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')

    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    with torch.no_grad():
        global attention   
        attention = model(**inputs)[-1]
    sentence_b_start = token_type_ids[0].tolist().index(1)
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)
    h = head_view([attention[attention_layer]], tokens, sentence_b_start, html_action='return')
    return h

def get_head_view_avg(segm, model, tokenizer):
    model = model.eval()
    text1, text2 = segm.split("$&*&*&$")
    inputs = tokenizer(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

    #sentence_a = "The cat sat on the mat"
    #sentence_b = "The cat lay on the rug"
    #inputs = tokenizer(sentence_a, sentence_b, return_tensors='pt')

    input_ids = inputs['input_ids']
    token_type_ids = inputs['token_type_ids']
    with torch.no_grad():
        attention = model(**inputs)[-1]
    sentence_b_start = token_type_ids[0].tolist().index(1)
    input_id_list = input_ids[0].tolist() # Batch index 0
    tokens = tokenizer.convert_ids_to_tokens(input_id_list)

    attention_mean = torch.Tensor()
    for l in attention:
        attention_mean = torch.cat((attention_mean, l), 0)
    attention_mean = torch.mean(attention_mean, 0, keepdim=True)

    h = head_view([attention_mean], tokens, sentence_b_start, html_action='return')
    return h

In [ ]:
#text1, text2 = respd[0][0].split("$&*&*&$")
text1, text2 = respd['text'][0].split("$&*&*&$")
inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

with torch.no_grad():
    global attention   
    attention = model1(**inputs)[-1]

In [ ]:
#inputs['input_ids'][0][128] #SEP
tokenizer1.decode(inputs['input_ids'][0][165])

In [ ]:
head4 = attention_mean[0][3][0]

In [ ]:
head1 = attention[4][0][0][0]

In [ ]:
sorted, indices = torch.sort(head1, descending=True)
print(indices[:10], sorted[:10])
#for i in indices[:5]:
#    print(tokenizer1.decode(inputs['input_ids'][0][i]))

In [ ]:
import numpy as np

attention_mean = torch.Tensor()
for l in attention:
    attention_mean = torch.cat((attention_mean, l), 0)
attention_mean = torch.mean(attention_mean, 0, keepdim=True)



In [ ]:
h00f= get_head_view(segm00, model1, tokenizer1, 11)

In [ ]:
with open("h00cu.html", "w") as file:
    file.write(h.data)

In [ ]:
attention[11].shape

In [ ]:
from collections import Counter

def attention_per_layer(attention, layer, threshold = 0.01, mode='set'):
    """ For a given attention matrix, get the list of tokens with strongest attention from CLS
    in a given layer in all attention heads
    Parameters
    threshhold : float
        The minimal value that attention from CLS to a certain token should posess to be included into the list
        Value around 0.02 only selects most sturdy links and in practice is mst common towards [SEP] token
        Value around 0.01 selects most tokens with noticeably visualizable attention
        Value around 0.15 filters the list, favouring more important tokens, yet creates a broad and meaningful list
    """
    attended_by_cls = []

    for headnum, head in enumerate(attention[layer][0]):
        #print(headnum)
        cls = head[0]
        values, positions = torch.sort(cls, descending=True)
        #for index, word in enumerate(positions[:10]):
        for index, word in enumerate(positions):
            if values[index] >= threshold:
                attended_by_cls.append(tokenizer1.decode(inputs['input_ids'][0][word]))
    if mode == 'set':
        return set(attended_by_cls) #to proceed with counting jointly for all instances
    if mode == 'counter':
        count_attention = Counter(attended_by_cls) #count separately for each instance
        count_attention = count_attention.most_common()
        return count_attention

def attention_general(attention, threshold = 0.01, mode='set'):
    """ Get the list of tokens with strongest attention from CLS in all attention heads, for all layers, represented as
    a list of per-layer lists
    """
    attended_general = []
    for layernum, layer in enumerate(attention):
        #print(layernum)
        attended_at_layer = attention_per_layer(attention, layernum, threshold, mode)
        #print(attended_at_layer)
        attended_general.append(attended_at_layer)
    return attended_general

In [ ]:
def attention_at_head(attention, layer, head, threshold = 0.01):
    attended_by_cls = []
    cls = attention[layer][0][head][0]
    values, positions = torch.sort(cls, descending=True)
    #print(positions)
    for index, word in enumerate(positions):
        #print(str(values[index]) + ": " + tokenizer1.decode(inputs['input_ids'][0][word]))
        if values[index] >= threshold:
            attended_by_cls.append([tokenizer1.decode(inputs['input_ids'][0][word]), values[index], word])
    return(attended_by_cls)


In [ ]:
def reconstruct_token(inputs, position, trackname = True):
    curpos = position
    startpos = position
    endpos = position

    if "##" in tokenizer1.decode(inputs['input_ids'][0][curpos]):
        while "##" in tokenizer1.decode(inputs['input_ids'][0][curpos]):
            curpos -= 1
        startpos = curpos
        curpos = position
        while "##" in tokenizer1.decode(inputs['input_ids'][0][curpos]):
            curpos += 1
        endpos = curpos
        
    elif "##" in tokenizer1.decode(inputs['input_ids'][0][curpos + 1]):
        curpos += 1
        while "##" in tokenizer1.decode(inputs['input_ids'][0][curpos]):
            curpos += 1
        endpos = curpos

    else:
        endpos += 1

    if trackname:
        word = tokenizer1.decode(inputs['input_ids'][0][startpos:endpos])
        if checkname(word):
            return(tokenizer1.decode(inputs['input_ids'][0][startpos:endpos]))
        else:
            return(tokenizer1.decode(inputs['input_ids'][0][position]))
    else:
        return(tokenizer1.decode(inputs['input_ids'][0][startpos:endpos]))

In [ ]:
def checkname(word):
    return True if word[0].isupper() else False
   

In [ ]:
a = reconstruct_token(inputs, 48, trackname=False)
print(a)
print(checkname(a))


In [ ]:
for index, i in enumerate(range(len(inputs['input_ids'][0]))):
    print(index, tokenizer1.decode(inputs['input_ids'][0][i]))

In [ ]:
a = attention_at_head(attention, 10, 0)
print(a)

filtered_imptokens = []
for imptoken in a:
    if imptoken[0] not in ['[SEP]', '[CLS]']:
        imptoken[0] = reconstruct_token(inputs, int(imptoken[2]), trackname = False)
        filtered_imptokens.append(imptoken)
print(filtered_imptokens)

In [ ]:
atan = pd.read_csv("Attention analysis.csv", sep=';')
atan_from = atan.loc[atan["Direction"] == "From CLS"]

extract_attention_per_instance(attention, inputs):
filtered_attention_per_instance = []
imptokens_per_instance = []
for index, head in enumerate(atan_from["Head"]):
    layer = int(atan_from.iloc[[index]]["Layer"])
    #print(layer, head)
    cur_attention_at_head = attention_at_head(attention, layer-1, head-1, threshold = 0.010)
    filtered_attention_at_head = []
    imptokens_at_head = []
    for imptoken in cur_attention_at_head:
        if imptoken[0] not in ['[SEP]', '[CLS]']:
            imptoken[0] = reconstruct_token(inputs, int(imptoken[2]), trackname = False)
            filtered_attention_at_head.append(imptoken)
            imptokens_at_head.append(imptoken[0])
    count_attention = Counter(imptokens_at_head) #count separately for each instance
    count_attention = count_attention.most_common()
    #print(filtered_attention_at_head)
    #print(len(imptokens_at_head), count_attention)
    imptokens_per_instance += imptokens_at_head
    filtered_attention_per_instance += filtered_attention_at_head

bow_attention_dict_per_instance = dict()
for imptoken in filtered_attention_per_instance:
    if imptoken[0] in bow_attention_dict_per_instance:
        bow_attention_dict_per_instance[imptoken[0]] += float(imptoken[1])
    else:
        bow_attention_dict_per_instance[imptoken[0]] = float(imptoken[1])
#print(Counter(imptokens_per_instance).most_common())
#print(attention_dict_per_instance)

"""attention_dict_per_instance = dict()
for imptoken in filtered_attention_per_instance:
    if imptoken[0] in attention_dict_per_instance:
        attention_dict_per_instance[imptoken[0]+'@@'+str(int(imptoken[2]))] += float(imptoken[1])
    else:
        attention_dict_per_instance[imptoken[0]+'@@'+str(int(imptoken[2]))] = float(imptoken[1])"""

apd = pd.DataFrame(bow_attention_dict_per_instance, index=[0])
apd = apd.transpose().sort_values(by=[0], ascending = False)
print(apd.head(10))
print(Counter(imptokens_per_instance).most_common())

acpd = pd.DataFrame(Counter(imptokens_per_instance), index=[0])
acpd = acpd.transpose().sort_values(by=[0], ascending = False)
print(acpd.head(10))

In [ ]:
def extract_attended_tokens_per_instance(attention, inputs):
    filtered_attention_per_instance = []
    imptokens_per_instance = []
    for index, head in enumerate(atan_from["Head"]):
        layer = int(atan_from.iloc[[index]]["Layer"])
        #print(layer, head)
        cur_attention_at_head = attention_at_head(attention, layer-1, head-1, threshold = 0.010)
        filtered_attention_at_head = []
        imptokens_at_head = []
        for imptoken in cur_attention_at_head:
            if imptoken[0] not in ['[SEP]', '[CLS]']:
                imptoken[0] = reconstruct_token(inputs, int(imptoken[2]), trackname = False)
                filtered_attention_at_head.append(imptoken)
                imptokens_at_head.append(imptoken[0])

        imptokens_per_instance += imptokens_at_head
        filtered_attention_per_instance += filtered_attention_at_head

    bow_attention_dict_per_instance = dict()
    for imptoken in filtered_attention_per_instance:
        if imptoken[0] in bow_attention_dict_per_instance:
            bow_attention_dict_per_instance[imptoken[0]] += float(imptoken[1])
        else:
            bow_attention_dict_per_instance[imptoken[0]] = float(imptoken[1])

    apd = pd.DataFrame(list(bow_attention_dict_per_instance.items()), columns = ['Feature','Weight'])
    apd = apd.sort_values(by=['Weight'], ascending = False).reset_index(drop=True)
    #print(apd.head(10))

    token_counter_per_instance = Counter(imptokens_per_instance)
    token_counter_per_instance = dict(token_counter_per_instance)

    acpd = pd.DataFrame(list(token_counter_per_instance.items()), columns = ['Feature','Counts'])
    acpd = acpd.sort_values(by=['Counts'], ascending = False).reset_index(drop=True)
    #print(acpd.head(10))
    return apd, acpd, imptokens_per_instance


In [ ]:
def extract_attended_tokens_per_instance(attention, inputs):
    filtered_attention_per_instance = []
    imptokens_per_instance = []
    for index, head in enumerate(atan_from["Head"]):
        layer = int(atan_from.iloc[[index]]["Layer"])
        #print(layer, head)
        cur_attention_at_head = attention_at_head(attention, layer-1, head-1, threshold = 0.010)
        filtered_attention_at_head = []
        imptokens_at_head = []
        for imptoken in cur_attention_at_head:
            if imptoken[0] not in ['[SEP]', '[CLS]']:
                imptoken[0] = reconstruct_token(inputs, int(imptoken[2]), trackname = False)
                filtered_attention_at_head.append(imptoken)
                imptokens_at_head.append(imptoken[0])

        imptokens_per_instance += imptokens_at_head
        filtered_attention_per_instance += filtered_attention_at_head

    bow_attention_dict_per_instance = dict()
    for imptoken in filtered_attention_per_instance:
        if imptoken[0] in bow_attention_dict_per_instance:
            bow_attention_dict_per_instance[imptoken[0]] += float(imptoken[1])
        else:
            bow_attention_dict_per_instance[imptoken[0]] = float(imptoken[1])

    mean_attention_dict_per_instance = dict()
    counts_of_attention_per_instance = dict()
    for imptoken in filtered_attention_per_instance:
        if imptoken[0] in mean_attention_dict_per_instance:
            mean_attention_dict_per_instance[imptoken[0]] += float(imptoken[1])
            counts_of_attention_per_instance[imptoken[0]] += 1
        else:
            mean_attention_dict_per_instance[imptoken[0]] = float(imptoken[1])
            counts_of_attention_per_instance[imptoken[0]] = 1

    for key in mean_attention_dict_per_instance.keys():
        mean_attention_dict_per_instance[key] = mean_attention_dict_per_instance[key] / counts_of_attention_per_instance[key]

    apd = pd.DataFrame(list(bow_attention_dict_per_instance.items()), columns = ['Feature','Weight'])
    apd = apd.sort_values(by=['Weight'], ascending = False).reset_index(drop=True)
    #print(apd.head(10))

    token_counter_per_instance = Counter(imptokens_per_instance)
    token_counter_per_instance = dict(token_counter_per_instance)

    acpd = pd.DataFrame(list(token_counter_per_instance.items()), columns = ['Feature','Counts'])
    acpd = acpd.sort_values(by=['Counts'], ascending = False).reset_index(drop=True)
    #print(acpd.head(10))

    apd_mean = pd.DataFrame(list(mean_attention_dict_per_instance.items()), columns = ['Feature','Weight'])
    apd_mean = apd_mean.sort_values(by=['Weight'], ascending = False).reset_index(drop=True)

    return apd, acpd, apd_mean #imptokens_per_instance


In [ ]:
extract = extract_attended_tokens_per_instance(attention, inputs)

In [ ]:
extract[0]

In [ ]:
#orig_logits = pipeline_onesegment(text, mode='logits')

def permute(segm, orig_logits, word):
        
    text1, text2 = segm.split("$&*&*&$")
    text1 = text1.replace(word, "")
    text2 = text2.replace(word, "")
    segm_changed = text1 + "$&*&*&$" + text2
    logits_changed = pipeline_onesegment(segm_changed, mode='logits')
    weight = np.subtract(orig_logits[0], logits_changed[0])

    switch = False

    if np.argmax(orig_logits[0]) != np.argmax(logits_changed[0]): 
        switch = True

    weight = list(weight)
    weight.append(switch)

    return weight

def permute_names(segm, orig_logits, word):
    top_weights = []
    top_switch = []
    #change_importance = []

    if checkname(word):
        changes = ["", word.upper(), word.lower(), word + " " + word, "John", "Mary", "Rinoa", "he", "she", "the person", "the Boy"]

        for change in changes:
            segm_changed = segm.replace(word, change)
            logits_changed = pipeline_onesegment(segm_changed, mode='logits')
            #print(orig_logits[0], type(orig_logits[0]), logits_changed[0], type(logits_changed[0]))
            weight = np.mean(np.abs(np.subtract(orig_logits[0], logits_changed[0])))

            switch = False

            #print(np.argmax(orig_logits[0]), np.argmax(logits_changed[0]))
            if np.argmax(orig_logits[0]) != np.argmax(logits_changed[0]): 
                switch = True

            #weight = [weight]
            #weight.append(change)
            top_weights.append(weight)
            top_switch.append(switch)
            #change_importance.append(weight[0])
    return top_weights, top_switch


In [ ]:
sample_indices = [4433, 7582, 2457, 2291, 6345, 6737, 2662, 3839, 5039, 7304, 6388, 2813, 2869, 6150, 2665, 1756, 4589, 
6286, 4001, 1596]

In [ ]:
sample_indices = [4433]

In [ ]:
names_importance_list

In [ ]:
filter_for_names = [7, 12, 13, 14, 15, 16, 17, 22, 24, 25, 27, 28, 29, 34, 35, 39, 40, 42, 43, 48, 49, 53, 54, 55, 56, 57, 62, 63, 64, 66, 73, 78, 79, 83, 87, 88, 89, 92, 
95, 96, 97, 99, 100, 102, 103, 104, 107, 108, 109, 112, 113, 114, 115, 117, 120, 122, 123, 126, 127, 128, 131, 132, 133, 138, 140, 143, 153,  
156, 159, 160, 161, 162, 168, 175, 176, 185, 189, 193, 194, 195, 205, 208, 218, 225, 240, 241, 245, 246, 247, 252, 253, 254, 255, 256, 
257, 263, 264, 265, 266, 270, 271, 272, 273, 274, 280, 281, 283, 284, 286, 287, 288, 292, 293, 297, 299, 300, 301, 302, 303, 304, 307, 312, 313, 
315, 316, 318, 359, 360, 361, 365, 370, 371, 374, 378, 387, 390, 392, 393, 399, 400, 404, 415, 417, 419, 420, 425, 426, 429, 432, 433, 434, 437, 
442, 443, 446, 447, 450, 453, 456, 457, 458, 461, 464, 465, 468, 471, 473, 477, 478, 481, 485, 489, 490, 492, 497, 499, 502, 503, 
504, 505, 506, 509, 510, 511, 512, 513, 514, 515, 516, 520, 521, 525, 527, 528, 537, 544, 545, 546, 547, 562, 567, 568, 570, 577, 578,
581, 593, 594, 595, 596, 598, 599, 601, 602, 603, 604, 606, 608, 609, 611, 612, 615, 616, 618, 619, 620, 622, 623, 626, 627, 628, 629, 630,
631, 632, 634, 635, 636, 638, 640, 641, 649, 652, 656, 657, 658, 659, 665, 673, 674, 680, 682, 686, 687, 688, 689, 690, 693,
701, 703, 705, 706, 708, 709, 711, 716, 717, 718, 719, 720, 721, 730, 737, 760, 770, 776, 778, 779, 780, 781, 784, 789, 790, 796]

In [ ]:
names_importance_list_filtered = []

for nameindex, name in enumerate(names_importance_list):
    if nameindex not in filter_for_names:
        names_importance_list_filtered.append(name)


In [ ]:
names_switches_list_filtered = []

for nameindex, name in enumerate(names_switches_list):
    if nameindex not in filter_for_names:
        names_switches_list_filtered.append(name)

In [ ]:
namesswpd_filtered = pd.DataFrame(names_switches_list_filtered, 
                        columns = ['Name','Del','UPPER','lower','dupl','john','mary','rinoa','he','she','person','boy'])
print(namesswpd_filtered.head(10))
for c in namesswpd_filtered.columns:
    if c != 'Name':
        print(c, 1 - (namesswpd_filtered[c].value_counts()[False] / len(namesswpd_filtered)))

In [ ]:
namespd_filtered = pd.DataFrame(names_importance_list_filtered, 
                        columns = ['Name','Del','UPPER','lower','dupl','john','mary','rinoa','he','she','person','boy'])
print(namespd_filtered.head(10))
for c in namespd_filtered.columns:
    if c != 'Name':
        print(c, namespd_filtered[c].mean(), namespd_filtered[c].median(), namespd_filtered[c].std())

In [ ]:
from matplotlib import pyplot as plt

fig7, ax7 = plt.subplots( figsize=(15, 15))


ax7.set_title('Intensity of prediction alteration for different changes in input names')
for c in namespd_filtered.columns:

    ax7.boxplot([namespd_filtered['Del'], namespd_filtered['UPPER'], namespd_filtered['lower'],
    namespd_filtered['dupl'], namespd_filtered['john'], namespd_filtered['mary'], namespd_filtered['rinoa'],
    namespd_filtered['he'], namespd_filtered['she'], namespd_filtered['person'], namespd_filtered['boy']], notch=True)
plt.xticks([1,2,3,4,5,6,7,8,9,10,11],['Del','Upper','Lower','Duplicate','John','Mary','Rinoa','He','She','The person','The boy'])
plt.yticks(np.arange(0, 12, 1))

plt.show()

In [ ]:
namespd = pd.DataFrame(names_importance_list, 
                        columns = ['Name','Del','UPPER','lower','dupl','john','mary','rinoa','he','she','person','boy'])
print(namespd.head(10))
for c in namespd.columns:
    if c != 'Name':
        print(c, namespd[c].mean(), namespd[c].std())

In [ ]:
namesswpd = pd.DataFrame(names_switches_list, 
                        columns = ['Name','Del','UPPER','lower','dupl','john','mary','rinoa','he','she','person','boy'])
print(namesswpd.head(10))

for c in namesswpd.columns:
    if c != 'Name':
        print(c, 1 - (namesswpd[c].value_counts()[False] / len(namesswpd)))

In [ ]:
subsamples = [0, 5, 10, 15, 20]
#res = []
names_importance_list = []
names_switches_list = []

atan = pd.read_csv("Attention analysis.csv", sep=';')
atan_from = atan.loc[atan["Direction"] == "From CLS"]

for count, textindex in enumerate(sample_indices):

    label = df['labels'][textindex]
    for s in subsamples:
        #cur_res = []

        correct = False
        segm = combine_segments_from_pd(textindex, s, 0) 
        #print(segm)
        prediction = pipeline_onesegment(segm, mode='labels')
        prediction = prediction[0]
        orig_logits = pipeline_onesegment(segm, mode='logits')
        if prediction == label:
            correct = True

        #cur_res += textindex, s, segm, label, correct, orig_logits

        
        text1, text2 = segm.split("$&*&*&$")
        inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

        with torch.no_grad():
            attention = model1(**inputs)[-1]
        
        top_features_sum, top_features_count, top_features_list = extract_attended_tokens_per_instance(attention, inputs)

        for index, feature in enumerate(top_features_sum['Feature'][:20]):

            if checkname(feature):
                print(feature)
                permutation, switches = permute_names(segm, orig_logits, feature)

                names_importance_list.append([feature]+permutation)
                names_switches_list.append([feature]+switches)
names_importance_list

In [ ]:
subsamples = [0, 5, 10, 15, 20]
all_features_list = []

atan = pd.read_csv("Attention analysis.csv", sep=';')
atan_from = atan.loc[atan["Direction"] == "From CLS"]

for count, textindex in enumerate(sample_indices):

    label = df['labels'][textindex]
    for s in subsamples:
 
        segm = combine_segments_from_pd(textindex, s, 0) 
        
        text1, text2 = segm.split("$&*&*&$")
        inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

        with torch.no_grad():
            attention = model1(**inputs)[-1]
        
        top_features_sum, top_features_count, top_features_list = extract_attended_tokens_per_instance(attention, inputs)

        all_features_list += top_features_list

        

In [ ]:
res_att[0][6]

In [ ]:
intersections1 = []
for index, i in enumerate(res_perc): 
    cur_intersection = []
    intersection = set(res_perc[index][6]).intersection(set(res[index][6]))
    percentage =  len(intersection) / len(set(res[index][6]))
    cur_intersection.append(set(res_perc[index][6]).intersection(set(res[index][6])))
    cur_intersection.append(percentage)
    intersections1.append(cur_intersection)

intersections2 = []
for index, i in enumerate(res_perc): 
    cur_intersection = []
    intersection = set(res_perc[index][7]).intersection(set(res[index][6]))
    percentage =  len(intersection) / len(set(res[index][6]))
    cur_intersection.append(set(res_perc[index][7]).intersection(set(res[index][6])))
    cur_intersection.append(percentage)
    intersections2.append(cur_intersection)

intersections3 = []
for index, i in enumerate(res_perc): 
    cur_intersection = []
    intersection = set(res_perc[index][8]).intersection(set(res[index][6]))
    percentage =  len(intersection) / len(set(res[index][6]))
    cur_intersection.append(set(res_perc[index][8]).intersection(set(res[index][6])))
    cur_intersection.append(percentage)
    intersections3.append(cur_intersection)

In [ ]:
intersections_pd1 = pd.DataFrame(intersections1, columns = ['Intersection','Percent'])
intersections_pd2 = pd.DataFrame(intersections2, columns = ['Intersection','Percent'])
intersections_pd3 = pd.DataFrame(intersections3, columns = ['Intersection','Percent'])
mean1 = intersections_pd1['Percent'].mean()
mean2 = intersections_pd2['Percent'].mean()
mean3 = intersections_pd3['Percent'].mean()

In [ ]:
logits_stat = []

for index, i in enumerate(res):
    #logits_stat.append(res[index][5][0][0])
    logits_stat.append(np.mean([np.abs(res[index][5][0][0]), np.abs(res[index][5][0][1])]))

logits_stat_pd1 = pd.DataFrame(logits_stat, columns = ['Logits'])
mean4 = logits_stat_pd1['Logits'].mean()




In [ ]:
intersections_pd1['Percent'].mean()

In [ ]:
LIME_no_Att = []
Att_no_LIME = []
for index, i in enumerate(res_att): 
    cur_intersection = []
    intersection = set(res_att[index][6]).intersection(set(res[index][6]))
    #percentage =  len(intersection) / len(set(res[index][6]))
    cur_intersection.append(set(res[index][6]) - intersection)
    #cur_intersection.append(percentage)
    LIME_no_Att.append(cur_intersection)
    cur_intersection = []
    cur_intersection.append(set(res_att[index][6]) - intersection)
    Att_no_LIME.append(cur_intersection)

In [ ]:
intersections_pd['Percent'].mean()

In [ ]:
resattpd['mean_switch'].mean()

# 0.06799999999999996 -- Count


In [ ]:
lime_features = pd.DataFrame(lime_features)

In [ ]:
from matplotlib import pyplot as plt

plt.figure(figsize=(30, 10))
plt.bar(range(len(diff_sort)), diff_sort, width=1, edgecolor=(0, 0, 0))

plt.xlabel("Features")
plt.ylabel("Average weight")
plt.title("Absolute difference in feature weights between classes, sorted")
plt.legend(fontsize=12)
plt.show()

In [ ]:
att = Counter(lime_features)

In [ ]:
att_features = pd.DataFrame(att_features)
att_features.to_csv("att_features.csv")

In [ ]:
att_features = []

for i in res_perc:
    att_features += i[7]

In [ ]:
mean_imp = []

for i in res:
    mean_imp.append(np.mean(i[8]))

mean_imp_pd = pd.DataFrame(mean_imp, columns = ['Mean'])
mean_imp[0] = mean_imp_pd['Mean'].mean()

# 0.8004299402236938 -- Count
# 0.7821930646896362 -- Sum
# 0.588703989982605 -- Mean
# 0.982 -- LIME

In [ ]:
max_imp = []

for i in res:
    max_imp.append(np.max(i[8]))

    
max_imp_pd = pd.DataFrame(max_imp, columns = ['Max'])
#max_imp[0] = max_imp_pd['Max'].mean()

#4.490159034729004 -- Count
#4.346518039703369 -- Sum
#3.6743555068969727 -- Mean
#3.626 -- LIME

In [ ]:
switch_count = 0
for i in res_att:
    if i[8]:
        switch_count += 1
switch_count

# 50 -- Count
# 49 -- Sum
# 42 -- Mean

In [ ]:
most_imp_tokens = []
for i in res_att:
    max_imp = max(i[9])
    max_index = i[9].index(max_imp)
    most_imp_tokens.append(i[6][max_index])

most_imp_tokens_counter = Counter(most_imp_tokens)
most_imp_tokens_counter = dict(most_imp_tokens_counter.most_common())
most_imp_tokens_pd = pd.DataFrame(list(most_imp_tokens_counter.items()), columns = ['Feature','Counts'])


    

In [ ]:
lanterns = []

for i in res_att:
    if "Lanterns" in i[2]:
        lanterns.append(i[2])
        

In [ ]:
subsamples = [0, 5, 10, 15, 20]
res_perc = []

atan = pd.read_csv(r"..\BertAA_content\Attention analysis.csv", sep=';')
atan_from = atan.loc[atan["Direction"] == "From CLS"]

for count, textindex in enumerate(sample_indices):

    label = df['labels'][textindex]
    for s in subsamples:
        cur_res = []

        correct = False
        segm = combine_segments_from_pd(textindex, s, 0) 
        #print(segm)
        prediction = pipeline_onesegment(segm, mode='labels')
        prediction = prediction[0]
        orig_logits = pipeline_onesegment(segm, mode='logits')
        if prediction == label:
            correct = True

        cur_res += textindex, s, segm, label, correct, orig_logits

        
        text1, text2 = segm.split("$&*&*&$")
        inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

        with torch.no_grad():
            attention = model1(**inputs)[-1]
        
        #top_features_sum, top_features_count, top_features_list = extract_attended_tokens_per_instance(attention, inputs)
        top_features_sum, top_features_count, top_features_mean = extract_attended_tokens_per_instance(attention, inputs)


        
        top_wordlist1 = []
        top_wordlist2= []
        top_wordlist3 = []

        for index, feature in enumerate(top_features_sum['Feature'][:20]):
            top_wordlist1.append(feature)
        for index, feature in enumerate(top_features_count['Feature'][:20]):
            top_wordlist2.append(feature)
        for index, feature in enumerate(top_features_mean['Feature'][:20]):
            top_wordlist3.append(feature)
        

        cur_res += top_wordlist1, top_wordlist2, top_wordlist3


        res_perc.append(cur_res)

In [ ]:
subsamples = [0, 5, 10, 15, 20]
res_att = []
all_features_list = []

atan = pd.read_csv(r"..\BertAA_content\Attention analysis.csv", sep=';')
atan_from = atan.loc[atan["Direction"] == "From CLS"]

for count, textindex in enumerate(sample_indices):

    label = df['labels'][textindex]
    for s in subsamples:
        cur_res = []

        correct = False
        segm = combine_segments_from_pd(textindex, s, 0) 
        #print(segm)
        prediction = pipeline_onesegment(segm, mode='labels')
        prediction = prediction[0]
        orig_logits = pipeline_onesegment(segm, mode='logits')
        if prediction == label:
            correct = True

        cur_res += textindex, s, segm, label, correct, orig_logits

        
        text1, text2 = segm.split("$&*&*&$")
        inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')

        with torch.no_grad():
            attention = model1(**inputs)[-1]
        
        #top_features_sum, top_features_count, top_features_list = extract_attended_tokens_per_instance(attention, inputs)
        top_features_sum, top_features_count, top_features_mean = extract_attended_tokens_per_instance(attention, inputs)

        top_wordlist = []
        top_attention = []
        top_weights = []
        switches = []
        mean_change = []
        for index, feature in enumerate(top_features_sum['Feature'][:20]):
        #for index, feature in enumerate(top_features_mean['Feature'][:20]):
                
            permutation = permute(segm, orig_logits, feature)

            top_wordlist.append(feature)
            #top_attention.append({feature: top_features_sum.iloc[[index]]['Weight']})
            #top_weights.append({feature: permutation})
            top_attention.append(top_features_sum.iloc[[index]]['Weight'])
            top_weights.append([permutation[0:2]])
            switches.append(permutation[2])
            mean_change.append(np.mean([np.abs(permutation[0]), np.abs(permutation[1])]))
            #else:
            #    continue
        
        percent_switches = switches.count(True) / len(switches)

        cur_res += top_wordlist, top_weights, percent_switches, mean_change

        all_features_list += top_wordlist

        res_att.append(cur_res)

In [ ]:
all_features_count1 = Counter(all_features_list1)
all_features_count1.most_common()

In [ ]:
all_features_count = Counter(all_features_list)
all_features_count.most_common()

In [ ]:
resattpd = pd.DataFrame(res_att, columns = ['index', 'segment', 'text', 'label', 'correctness', 'logits',  'topwords', 'topwords_change', "mean_switch", 'mean_change'])

In [ ]:
resattpd.to_csv('res_att_100.csv', index=False)

In [ ]:
permutation = permute(segm, orig_logits, feature)
permutation

In [ ]:
change = np.mean([np.abs(permutation[0]), np.abs(permutation[1])])
change

In [ ]:
res[0][9]



In [ ]:
p = permute(text, orig_logits, "Sara")
print(p)

In [ ]:
for feature in acpd.index[:10]:
    print(feature)

In [ ]:
acpd[:10]

In [ ]:
p = permute_names(text, orig_logits, "Sara")
print(p)

In [ ]:
p[1]

In [ ]:
p[1]

In [ ]:
for i in inputs:
    print(i)

In [ ]:
text

In [ ]:
a = attention_per_layer(attention, 0, threshold = 0.015, mode="counter")
a

In [ ]:
attentions = []
for text in respd['text']:
    text1, text2 = text.split("$&*&*&$")
    inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')
    with torch.no_grad():
        attention = model1(**inputs)[-1]
    cls_attention = attention_general(attention, threshold = 0.01, mode='counter')
    attentions += [cls_attention]

attentions_pd = pd.DataFrame(attentions)

In [ ]:
set_attentions = []
for text in respd['text']:
    text1, text2 = text.split("$&*&*&$")
    inputs = tokenizer1(text1, text2, truncation=True, padding='max_length', max_length=255, return_tensors='pt')
    with torch.no_grad():
        attention = model1(**inputs)[-1]
    cls_attention = attention_general(attention)
    set_attentions += [cls_attention]

set_attentions_pd = pd.DataFrame(set_attentions)

In [ ]:
allattset = []
for attset in set_attentions_pd[11]:
    allattset += list(attset)

count = Counter(allattset)
count = count.most_common()
count

#set separately for each layer, but for all heads, for all instances in our sample


In [ ]:
attended_by_cls = []

for layernum, layer in enumerate(attention):
    #print(layernum)
    for headnum, head in enumerate(layer[0]):
        #print(headnum)
        cls = head[0]
        sorted, indices = torch.sort(cls, descending=True)
        for word in indices[:10]:
            attended_by_cls.append(tokenizer1.decode(inputs['input_ids'][0][word]))

print(len(attended_by_cls))
from collections import Counter

count = Counter(attended_by_cls)
count = count.most_common()
count
        

In [ ]:
df.iloc[[4433]]

In [ ]:
df['labels'][4433]

In [ ]:
df['0'][1596]

In [ ]:
explainer = LimeTextExplainer(bow=True)
exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)

# Pipeline for collecting explanations

In [ ]:
sample_indices = [4433, 7582, 2457, 2291, 6345, 6737, 2662, 3839, 5039, 7304, 6388, 2813, 2869, 6150, 2665, 1756, 4589, 
6286, 4001, 1596, #20
 4817, 2261, 7292, 2595, 128, 3533, 2443, 6652, 3820, 796, 3518, 110, 575, 4142, 1950, 1216, 2084, 2534, 3291, 4807, 2269,
 3625, 1154, 5049, 5653, 5924, 2366, 3425, 1821, 3610, 348, 2181, 6432, 7560, 6981, 5382, 3898, 2889, 4019, 80, 
5578, 3515, 3151, 4652, 1838, 2447, 2319, 763, 2963, 1914, 5210, 5609, 2609, 3915, 4370, 6654, 5096, 3640, 3634, 3073, 3092, 
794, 695, 4275, 757, 4527, 5194, 421, 2121, 649, 2306, 3796, 4805, 6340, 2930, 3963, 7422, 4290, 736, 900]

In [ ]:
dataset = pd.read_json("..\BertAA_content\Data\pan20-authorship-verification-training-small.jsonl", lines=True)

In [ ]:
dataset.columns

In [ ]:
dataset.iloc[50285]

In [ ]:
for count, index in enumerate(sample_indices):
    id = df['3'][index]
    index = dataset.index[dataset['id'] == id]
    print(dataset['fandoms'][index])

In [ ]:
for count, index in enumerate(sample_indices):


In [ ]:
subsamples = [0, 5, 10, 15, 20]
res = []

explainer = LimeTextExplainer(bow=True)
explainerleft = MyLimeTextExplainer(bow=True, mode='left')
explainerright = MyLimeTextExplainer(bow=True, mode='right')

for count, index in enumerate(sample_indices):


    label = df['labels'][index]
    for s in subsamples:
        cur_res = []

        correct = False
        segm = combine_segments_from_pd(index, s, 0) 
        #print(segm)
        prediction = pipeline_onesegment(segm, mode='labels')
        prediction = prediction[0]
        orig_logits = pipeline_onesegment(segm, mode='logits')
        if prediction == label:
            correct = True

        cur_res += index, s, segm, label, correct, orig_logits

        """if s == 0:
            hcf = get_head_view_avg(segm, model1, tokenizer1)
            with open("hc{}f.html".format(index), "w") as file:
                file.write(hcf.data)
            hcu = get_head_view_avg(segm, model2, tokenizer2)
            with open("hc{}u.html".format(index), "w") as file:
                file.write(hcu.data)
            for layer in range(12):
                hf = get_head_view(segm, model1, tokenizer1, layer)
                with open("h{}_{}f.html".format(index, layer), "w") as file:
                    file.write(hf.data)
                hu = get_head_view(segm, model2, tokenizer2, 11)
                with open("h{}_{}u.html".format(index, layer), "w") as file:
                    file.write(hu.data)
        """
        if prediction == 0:
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_features = exp.as_list()
            top_wordlist = []
            top_weights = []
            for f in top_features:
                top_wordlist.append(f[0])
            for word in top_wordlist:
                segm_changed = segm.replace(word, "")
                logits_changed = pipeline_onesegment(segm_changed, mode='logits')
                #print(orig_logits[0], type(orig_logits[0]), logits_changed[0], type(logits_changed[0]))
                weight = np.subtract(orig_logits[0], logits_changed[0])
                mean_weight = np.mean([np.abs(weight[0]), np.abs(weight[1])])
                top_weights.append(mean_weight)

            """for word in top_wordlist:
                segm = segm.replace(word, "")
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_sec_features = exp.as_list()
            top_sec_wordlist = []
            for f in top_sec_features:
                top_sec_wordlist.append(f[0])"""

        if prediction == 1:

            exp_l = explainerleft.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=6, num_samples=5000)
            top_features_l = exp_l.as_list()
            top_wordlist = []
            top_weights = []
            for f in top_features_l:
                top_wordlist.append(f[0])

            exp_r = explainerright.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=6, num_samples=5000)
            top_features_r = exp_r.as_list()
            for f in top_features_r:
                top_wordlist.append(f[0])
                
            for word in top_wordlist:
                segm_changed = segm.replace(word, "")
                logits_changed = pipeline_onesegment(segm_changed, mode='logits')
                #print(orig_logits[0], type(orig_logits[0]), logits_changed[0], type(logits_changed[0]))
                weight = np.subtract(orig_logits[0], logits_changed[0])
                mean_weight = np.mean([np.abs(weight[0]), np.abs(weight[1])])
                top_weights.append(mean_weight)

            """for word in top_wordlist:
                segm = segm.replace(word, "")
            exp = explainer.explain_instance(text_instance=segm, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)
            top_sec_features = exp.as_list()
            top_sec_wordlist = []
            for f in top_sec_features:
                top_sec_wordlist.append(f[0])"""

        cur_res += top_wordlist, top_features, top_weights

        res.append(cur_res)

        

In [ ]:
res[0]

In [ ]:
segm = """" Sir, I have the the reports you"ve asked for" The bridge commander stated. " Ahhh,
 thank you commander. Prep the fleet for the jump to Kashyyik" Admiral Segutav 
ordered in his thick German accent. " Yessir. Setting location now" The commander replied. " if I may ask sir, why did you need the reports on the 
Ghost"s crew?" " 
I"ve been put in charge of this fleet by none other than Grand Admiral Thrawn himself. He put me in command of this fleet with the sole purpose to destroy the Phoenix Squadron.
$&*&*&$Hercules woke up when he heard Xena dreaming so he went
 and fetched some ale and put some sleeping poison in it. "Xena, here take a drink." Hercules kindly demanded. 
"Thank you, I think I need a drink." Xena replied. "Xena, what was that all about?" Hercules asked. 
"I had a dream I was on Mount Olympus and Ares jumped out from somewhere with Gabrielle, then we started talking and all of a sudden he killed Gabrielle." Xena sobbed."""

#segm = combine_segments_from_pd(4433, 0, 0) 
print(segm)

res = pipeline_onesegment(segm.replace("Segutav", ""), mode="logits")
res
#[ 4.5062966, -4.566842 ]
#[ 4.1582108, -4.244451 ]
#[-3.4521582,  2.2805903]

In [ ]:
respd = pd.DataFrame (res, columns = ['text', 'label', 'correctness', 'logits', 'topwords', 'topwords_lime', 'topwords_obf'])

In [ ]:
respd.to_csv('res_100.csv', index=False)

In [ ]:
res[0][4]

In [ ]:
predictions_onetext = pipeline_onetext(text4)

In [ ]:
def get_data_from_combined_texts_long(text_or_list):
    """
    Get data from raw text that contains two fragments and a separater, or from a list of texts,
    each of them containing two fragments and a separater. Used in pipeline_onetext. The ONLY type
    of data processor for LIME inputs
    """

    print("Getting data from raw text")

    datas = []

    print(type(text_or_list), len(text_or_list))
    if not isinstance(text_or_list, str):
        for text_variant in text_or_list:
            text1, text2 = text_variant.split("$&*&*&$")
            text1 = text_segmentate(text1, maxlen=750, seps='.?!;')
            text2 = text_segmentate(text2, maxlen=750, seps='.?!;')
            while len(text1) < 30 or len(text2) < 30:
                    if len(text1) < 30:
                        n_text1 = []
                        for i in range(30):
                            for sent in text1:
                                n_text1.append(sent)
                        text1 = n_text1
                    elif len(text2) < 30:
                        n_text2 = []
                        for i in range(30):
                            for sent in text2:
                                n_text2.append(sent)
                        text2 = n_text2
            datas.append((text1, text2))
    else:
        text1, text2 = text_or_list.split("$&*&*&$")
        text1 = text_segmentate(text1, maxlen=750, seps='.?!;')
        text2 = text_segmentate(text2, maxlen=750, seps='.?!;')
        while len(text1) < 30 or len(text2) < 30:
                if len(text1) < 30:
                    n_text1 = []
                    for i in range(30):
                        for sent in text1:
                            n_text1.append(sent)
                    text1 = n_text1
                elif len(text2) < 30:
                    n_text2 = []
                    for i in range(30):
                        for sent in text2:
                            n_text2.append(sent)
                    text2 = n_text2
        datas.append((text1, text2))
    return datas


In [ ]:
with open('textcomb3.txt', 'r') as text:
    text3 = text.read()

segments = get_data_from_combined_texts_long(text3)

In [ ]:
def combine_segments(index, sep_option=0, write=False):
    """
    Combine a pair of texts from dataset with a separator and turn into a single text
    """

    text1 = segments[0][0][index]
    text2 = segments[0][1][index]
    sep = "$&*&*&$" if sep_option == 0 else "[SEP]"
    text_combined = text1 + sep + text2
    
    if write:
        name = "textcomb{}.txt".format(index)
        with open(name, 'w') as textcomb:
            textcomb.write(text_combined)

    return(text_combined)

In [ ]:
segm2 = combine_segments(1)
segm2

In [ ]:
segm1 = combine_segments(0, 1)
segm1

In [ ]:
testtext16 = combine_segments_from_pd(0,16)
testtext16

In [ ]:
testtext0_changed

In [ ]:
testtext0_changed = """"OW!" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." MARY yelled as she continued to yell and jump about.
 "Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. "Wow your lighter 
 then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.$&*&*&$"Are you okay?" His simple inquiry about her 
 farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment. "What do you think?" her face now  the 
evidence of the strain she had been carrying for weeks. Her separation from Marcos made her achy and distressed. Her heart was breaking into a million
 pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

"""She convinced the guard to let her through the gates and she ran up the familiar stairs until she got to the potted bush next to the front door, she reached 
down and pulled out the spare key Joey kept there for her. She used them to unlock the door and then tossed her bag down, leaving the door open, and ran upstairs 
into his room. She then fell down on his bed, held onto a pillow and cried so hard that she didn"t even hear anyone enter the house. "Mary!" Joey yelled, standing 
dumb-struck in the doorway.$&*&*&$"Joh..n." her voice turned breathy, heat suffusing through her pores. A loud crash sounded behind them, echoing through the hall. 
They jerked apart and saw Marcos had accidentally dropped a cement block and created a hole in the floor while Lorna stood glaring at him. 
Then she bit out,"Nice going laser. As usual I"ll have to fix your mess." Then proceeded to maneuver a few metals plates through the hole in an attempt to mend it."""

In [ ]:
res = pipeline_onesegment(testtext16, mode='logits')
res

In [ ]:
testtext0_changed = testtext16.replace("Clarice","")


res = pipeline_onesegment(testtext0_changed, mode='logits')
print(res)


#example 0-0 
#[[ 4.7861366 -4.759954 ]] class 0 correct confidence 1
#['She', 'MARY', 'and', 'proceeded', 'ran', 'bush']
#without She (weight 0.28) [[ 4.1627555 -4.1556478]]
#without MARY (weight 0.24) [[ 4.9938765 -5.0127206]] contrary to Lime!
#if we replace MARY with Mary the difference becomes larger [[ 5.4506316 -5.633418 ]] which means that CASING MATTERS
#if we replace MARY with JOHN texts get even more different [[ 5.380384 -5.513166]]. Keep increasing slightly if we add more JOHN
#if we add JOHN to the second part as well, gets less different [[-4.439573  3.488577]]
#without all topwords [[-2.9259872  1.8621671]] LABEL CHANGES

#example 0-4
#[ 1.371373 , -1.5087308] class 0 correct confidence 0.95
#['bore', 'yelled', 'strain', 'worded', 'What', 'said']
# without bore (weight 0.15) label SUDDENLY CHANGES [[-2.2827718  1.4593749]]
# without worded (weight 0.14) label ALSO SUDDENLY CHANGES [[-1.9648905  1.2226754]]
# without farewell (weight 0.12) ALSO [[-2.2466617  1.45032  ]], same for touched, said
# and they somewhat combine: when we remove all 3 -- bigger changer [[-4.2394676  3.2303755]]
# while without yelled (weight 0.11) it only goes in another direction [[ 2.6601286 -2.9694483]]
# SO here the model very easily switches to class 1

#example 0-16
#[[-5.0536942,  4.766159 ]] class 1 incorrect confidence 1
# however, it's enough to remove 1 name. Without Clarice (weight 0.26) [[ 2.7647822 -3.1579168]]


In [ ]:
segm1 = """Rinoa let out a soft giggle. "Okay Uncle Laguna." "As always, make yourselves at home!" Kiros cleared his throat. "Laguna, I believe our guests are hungry." "OH! Yes yes, I"m sorry." 
Laguna scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed Laguna as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one, though the lenghty table was pleasingly decorated with foods on the table.$&*&*&$The crown prince already entreated help from the glaives, a last resort he would 
rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none.
 It was when nea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""

In [ ]:
segm3 = """"" Mary yelled, jumping around, trying very hard not to curse or swear. "Ow, mother flipping chicken poop." Mary yelled as she continued to yell and jump about. 
"Okay now it"s time for you to go to the hospital." Joey said, knocked her knees out and picked her up in order to carry her downstairs into the kitchen for ice. 
"Wow your lighter then I remember. Have you not been eating?" Joey asked, slightly alarmed. He could feel all of her bones through her clothes.$&*&*&$"Are you okay?" 
His simple inquiry about her farewell touched her more than a well-worded lecture would have. So she softened a little. Distress quickly morphing into a bout of self-pity and embarrassment.
 "What do you think?" her face now bore the evidence of the strain she had been carrying for weeks. 
Her separation from Marcos made her achy and distressed. Her heart was breaking into a million pieces every day as the distance between her and Marcos continued to widen. "I think you need to talk to him."""

In [ ]:
segm4 = """What hospital are you at?" Dustin said and there was a lot of rustling at the other end. "The one near my house, Clarice. I didn"t catch the name I was a little occupied. 
Mary was throwing a fit." Joey answered sincerely. "Kay, we are coming. See you in a few." Dustin said and the line went dead. Joey took a seat along the wall and waited for news about Mary.
 Just a tad bit of drama, hope its not too confusing. Blondie : P Oh gosh, sorry. I left you a cliff hanger. I wonder what will happen to Mary today.$&*&*&$Clarice bobbed her head slightly from side to side and with a sheepish look remarked,
  "Not exactly but something to the effect, Clarice. I did call him a ninny though." John flashed his killer smile, it made her gooey every time. In an instant his eyes turned intense again and he rested his forehead on hers. The change in him didn"t go unnoticed by Clarice. 
"What?" "Clarice I...feel it again." He had told her about how he sometimes felt a foreboding feeling. "Hey."""

In [ ]:
res = pipeline_onesegment(segm1, mode='logits')
res = [-5.531711 ,  5.7830925], [-5.3570204,  5.4185996]
#Without Ara
#shap prediction 1: 0.506, 0: -0.474
#in reality 1: 0.364, 0: -0.175

In [ ]:
res = pipeline_onesegment(segm4, mode='logits')
res
#[-5.0536942,  4.766159 ]
#One more Clarice [-5.1230087,  4.9003825]
#Two more Clarice[-5.2671213,  5.1808963]
#Third Clarice now in the first segment [-5.4629855,  5.5636225]

In [ ]:
res = pipeline_onesegment(segm3, mode='logits')
res
#[ 1.371373 , -1.5087308]
#OW [ 2.905075 , -3.2013812]
#! [ 2.2600317, -2.5298123]
#OW! [-0.00994844, -0.42599773]
#shap prediction OW for class 1 OW -0.305 ! -0.27
#shap prediction OW for class 0 OW 0.303 ! 0.308
#in reality for class 1 OW +1.54 ! +1.02 OW! -1.08
#in reality for class 0 OW -1.53 ! -0.89 OW! 1.38 


In [ ]:
segm1

## Get explanations for separate segments using pipeline_onesegment

In [ ]:
explainer = LimeTextExplainer(bow=True)
explainerleft = MyLimeTextExplainer(bow=True, mode='left')
explainerright = MyLimeTextExplainer(bow=True, mode='right')

In [ ]:
explainerchar = LimeTextExplainer(char_level=True)#, split_expression = r'[^A-Za-z.?!,"-]')

In [ ]:
exp16 = explainer.explain_instance(text_instance=testtext16, classifier_fn=pipeline_onesegment, num_features=10, num_samples=5000)

In [ ]:
exp16.show_in_notebook(text=True)
#lime simple, test example 0-16

In [ ]:
exp.show_in_notebook(text=True)
#lime simple, test example 0-4
exp4.show_in_notebook(text=True)
#lime simple, test example 0-4 without bore
exp4_2.show_in_notebook(text=True)
#lime simple, test example 0-4 with 10k samples

In [ ]:
exp.show_in_notebook(text=True)
#lime simple, test example 0-4

In [ ]:
exp.show_in_notebook(text=True)
#lime simple, test example 0-0

In [ ]:
top_features = exp.as_list()
top_list = []
for f in top_features:
    top_list.append(f[0])
top_list

In [ ]:
def remove_top(text, top):
    for word in top:
        text = text.replace(word, "")
    return text

In [ ]:
testtext0_changed = remove_top(testtext0, top_list)

In [ ]:
exp.show_in_notebook(text=True)
#main model, text without top features excluding Aranea

In [ ]:
exp.show_in_notebook(text=True)
#main model, text without top features including Aranea

In [ ]:
exppunct.show_in_notebook(text=True)
#with punctuation, simple lime

In [ ]:
exp.show_in_notebook(text=True)
#180000 model, simple lime

In [ ]:
expleft.show_in_notebook(text=True)
#180000 model, left lime

In [ ]:
expleft.show_in_notebook(text=True)
#main model, left lime

In [ ]:
expright.show_in_notebook(text=True)
#180000 model, right lime

In [ ]:
expright.show_in_notebook(text=True)
#main model, right lime

In [ ]:
exp2 = explainer2.explain_instance(text_instance=segm2, labels=(1,), classifier_fn=pipeline_onesegment, num_features=6)

In [ ]:
exp2.show_in_notebook(text=True)

In [ ]:
segm1

In [ ]:
res = pipeline_onesegment(segm1.replace('giggle','giggggggle'), mode='logits')
print(res)

#[-5.531711   5.7830925] 0 1
#Without Aranea:
#[-4.927722   4.5944533]
#Without Aranea and all top features (as in segm1_notop): Label changes
#[[ 1.4439765 -1.4362624]]

#Without Rinoa:
#[[-5.381693   5.3939533]]
#Without Rinoa and Laguna:
#[[-5.2352524  5.1498575]]
#and though
#[[-5.0144567  4.673427 ]]
#and Kiros
#[[-4.0238514  2.966291 ]]
#and OH: grows back!
#[[-4.8581166  4.3767138]]

#DEFAULT NOTOP without cleared instead of OH: significant drop (ALL top-features for class 1 removed)
#[[-2.8962233  1.7099165]]
# without cleared and OH: grows back
#[[-4.005427   2.9661796]]
#If we return OH but remove a quote before it -> LABLE CHANGES
#[[-0.14309846 -0.2922034 ]]
#If we remove a quote before OH without other changes -> only a tiny drop
#[[-5.5299683  5.7772665]]
#If we remove a quote AND OH after removing the top features -> lack of OH pushes back, nothing important
#[-4.096264   3.1056435]

# [[-5.5027714  5.6508436]] Rinoa removed and OH instead of "OH
# [[-5.498661  5.7157  ]] Laguna and
# [[-5.5193887  5.702038 ]] Kiros and
# [[-5.5278864  5.8021803]] though and
# [[-5.5328455  5.756836 ]] cleared and

# when "OH is turned into OH, other top features weight more. But when Laguna was the last, it didn't change much until we removed Laguna

#Removed some " -> label 1 grows a bit
#[[-5.5252357  5.802293 ]]
#Removed all " -> label 0 grows more
#[[-5.3608084  5.3494406]]

#Removed all " but also top features -> label changes and opposite logits are HUGE!
#[[ 2.9483943 -3.1187959]]



#replacing " with ' -> not significant; . with ! -> NS, ! with , or . -> NS; removing , NS

#after removing all top features, changing . with ! becomes a lot more important and now class 1 is a lot more probable
#[[-4.64954    4.0420275]]
#changing only in the first text doesn't matter much (even though 8 occurences)
#[[-2.8171382  1.7177728]]
#changing all . to ! in the second didn't matter much until we changed all ! to . in the first:
#[[ 2.222923  -2.2992437]]
#however, if we simply change all ! to . (that is, only change the first), we only get small increase of dissimilarity (although technically pieces become more uniform!)
#[[-2.6927602  1.4867294]]
# changing ! to ? instead of . is a bit more important
#[[-2.616405  1.396568]]
# removing . completely from one or both makes texts more similar
#[[-4.8784447  4.341698 ]]
# and it goes further when we remove other punctuation as well as .
#[[-5.441371   5.5717177]]

#it is EVEN MORE SALIENT with , if we only remove ,
#[[-5.252601   5.1758327]]

#getting in another directinon (label 0) is not as easy. We only found one case before

#adding new punctuations allows to do so rather fast. After adding a few extra . to the first text (only when also removing from second):
#[[ 1.8393167 -1.9136158]]
# When we kept the second text unchanged, the , didn't change the label
#[[-2.5794852  1.3692824]]
#however, as soon as we added , after I believe, the LABEL CHANGED!
#[[ 1.5631607 -1.6246212]]
# and changed even more when we added , after our guests instead (and they DIDN'T COMBINE!)
#[[ 1.7775549 -1.8065834]]
#not every change is so important. , after let out only changed the results slightly and didn't influence the label
#[[-2.0973256  1.0720851]]

In [ ]:
segm1_changed = segm1.replace('', '')

In [ ]:
segm1_changed = segm1_changed.replace('Laguna', '')

In [ ]:
segm1_changed = segm1_changed.replace('cleared', '')

In [ ]:
segm1_changed = segm1_changed.replace('though', '')

In [ ]:
segm1_changed = segm1_changed.replace('Kiros', '')

In [ ]:
segm1_changed = segm1_changed.replace('"OH', 'OH')

In [ ]:
segm1_changed = segm1_notop.replace('"! Yes', 'OH! Yes')

In [ ]:
segm1_changed = segm1_notop.replace('.', ',', 8).replace('!', '?', 3)

In [ ]:
segm1_changed

In [ ]:
segm1

In [ ]:
segm1_changed = """ let out a soft giggle. "Okay Uncle ." "As always, make yourselves at home!"   his throat. ", I believe our guests are hungry." "OH! Yes yes, I"m sorry."  
scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed  as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one,  the lenghty table was pleasingly decorated with foods on the table.$&*&*&$The crown prince already entreated help from the glaives, 
a last resort he would rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none. 
It was when Aranea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""

In [ ]:
segm1_changed = """ let out a soft giggle. "Okay Uncle ." "As always, make yourselves at home!"   his throat. ", I believe our guests are hungry." "OH! Yes yes, I"m sorry."  
scratched the back of his neck in embarassment. "To the dining hall we go!" They all followed  as he went through one of the sliding doors on the right side of the room. 
The dining hall was a plain one,  the lenghty table was pleasingly decorated with foods on the table.$&*&*&$The crown prince already entreated help from the glaives, 
a last resort he would rather not do as he did not want his father to be anymore involved. But they were heavily outnumbered, and their chances of surviving were slimming to none. 
It was when Aranea decided to show up with her own infantry jumping from her red ship. There was so much distrust towards her at first, knowing how long she had served the Emperor and carried out his orders."""

In [ ]:
def text_segmentate(text, maxlen, seps='\n', strips=None):
    """将文本按照标点符号划分为若干个短句
    """
    text = text.strip().strip(strips)
    if seps and len(text) > maxlen:
        pieces = text.split(seps[0])
        text, texts = '', []
        for i, p in enumerate(pieces):
            if text and p and len(text) + len(p) > maxlen - 1:
                texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
                text = ''
            if i + 1 == len(pieces):
                text = text + p
            else:
                text = text + p + seps[0]
        if text:
            texts.extend(text_segmentate(text, maxlen, seps[1:], strips))
        return texts
    else:
        return [text]

In [ ]:
segmented_4 = text_segmentate(text4, seps=".!?", maxlen=510)
print(segmented_4[0], len(segmented_4[0]), len(segmented_4[0].split(" ")))

## Get explanations using pipeline_onetext

In [ ]:
import lime
from lime import lime_text
from lime.lime_text import LimeTextExplainer
#explainer = LimeTextExplainer(bow=False)
#explainer2 = LimeTextExplainer()

In [ ]:
text4 = combine_texts(4)

In [ ]:
exp2.show_in_notebook(text=True)

In [ ]:
exp = explainer2.explain_instance(text_instance=text4, labels=(0,), classifier_fn=pipeline_onetext, num_features=6)

In [ ]:
exp2 = explainer2.explain_instance(text_instance=text4, labels=(0,1), classifier_fn=pipeline_onetext, num_features=6)

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp_true = explainer.explain_instance(text_instance=text_combined, classifier_fn=pipeline_onetext, num_features=6)

In [ ]:
exp_true3 = explainer.explain_instance(text_instance=text_combined, classifier_fn=pipeline_onetext, num_features=6)

In [ ]:
exp.show_in_notebook(text=True)


In [ ]:
exp.save_to_file('textcomb1.html')


In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp.show_in_notebook(text=True)

In [ ]:
exp.as_list()

In [ ]:
exp_true.show_in_notebook(text=True)

In [ ]:
exp_true.as_list()

In [ ]:
exp_true2.show_in_notebook(text=True)

In [ ]:
exp_true3.show_in_notebook(text=True)

## Get a certain prediction and save corresponding embedding for future analysis

In [ ]:
predictions_truth_uncertain = pipeline_onetext(text_combined)

In [ ]:
# the embedding is formed during the execution of pipeline_onetext

torch.save(embedding, "..\\BertAA_content\\predictions_truth_uncertain.pt")


## Analyze the input

In [ ]:
from collections import Counter

def vocab_size(text, k=15, r=0):
    """This functions calculates and prints the results for a text's vocabulary
    It returns vocabulary in terms of unique:
    1. Tokens
    2. Words or Numbers
    3. Words Only
    4. Words (less English stopwords)
    5. Complex Words > Length k (default = 15)"""

    #Default Option, Print
    if r == 0:
        #Unique or "u" Sets
        text = text.split()
        u_tokens = len(set(text))
        u_words_num = len(set([w for w in text if w.isalnum()]))
        u_words = len(set([w for w in text if w.isalpha()]))
        #u_words_less_stopwords = len(set([w for w in text if w.lower() not in stopwords.words('english')]))
        u_complex_words = len(set([w for w in text if w.isalnum() and len(w.lower()) > k ]))

        print ("__"*30, '\n',  \
               "The text has the following vocabulary: ", '\n', \
               "Unique Tokens: ", u_tokens, '\n', \
               "Unique Words or Numbers: ", u_words_num, '\n', \
               "Unique Words: ", u_words,'\n', \
               #"Unique Words (Less Stopwords): ", u_words_less_stopwords, '\n', \
               "Unique Complex Words (> ", k, " Characters): ", u_complex_words, '\n', \
               "__"*30 \
               )
        print(Counter([w for w in text if w.isalpha()]))

    #Option to Return Complex Words
    elif r == 1:
        ucw = [w.lower() for w in text if w.isalnum() and len(w.lower()) > k ]
        return ucw
    else:
        pass

In [ ]:
vocabtext = "textcomb.txt"

def stat_for_text(vocabtext, split=False):
    with open(vocabtext, 'r') as vocabtext:
        vocabtext = vocabtext.read()

        if split:
            text1, text2 = vocabtext.split("$&*&*&$")
            vocab_size(text1)
            print("\n"*2+"#"*60+"\n"*2)
            vocab_size(text2)
        else:
            vocab_size(vocabtext)

stat_for_text(vocabtext, split=True)

In [ ]:
explainer = MyLimeTextExplainer(bow=False, mode='right')

In [ ]:
explainer2 = MyLimeTextExplainer(bow=True, mode='left')

In [ ]:
exp = explainer2.explain_instance(text4, classifier_fn=pipeline_onetext, num_samples=5000, num_features=6)

In [ ]:
#5000 permutations, bow, left mode
exp.show_in_notebook(text=True)

In [ ]:
#5000 permutations, bow, right mode
exp.show_in_notebook(text=True)

In [ ]:
exp.intercept

# compare with ret_exp.intercept {1: 0.998998835482347}

In [ ]:
myprediction = pipeline_onetext(text4, mode='logits')

In [ ]:
myprediction1 = pipeline_onetext(text4, mode='logits')
myprediction2 = pipeline_onetext(text_4_no_aranea, mode='logits')
print(myprediction1, myprediction2)

In [ ]:
text_4_no_aranea = text4.replace('Aranea','').replace('Ignis','').replace('Gladiolus','').replace('Rinoa','').replace('Squall','')

In [ ]:
print(text_4_no_aranea)

In [ ]:
exp.show_in_notebook(text=True)
#left

In [ ]:
exp.show_in_notebook(text=True)
#right

In [ ]:
from lime.lime_text import TextDomainMapper


explainer.explain_instance(text4, classifier_fn=pipeline_onetext, num_samples=5000, labels=(1,))

In [ ]:
data,yss,distances=explainer._MyLimeTextExplainer__data_labels_distances(MyIndexedString(text4, bow = False),classifier_fn=pipeline_onetext,num_samples=5000,mode='rand')
## Top 2 closest perturbed samples
df=pd.DataFrame(distances,columns=['distance'])
df1=df.sort_values(by='distance')
req_index=df.index[1:50]
closest_perturbed_sample=[]
for k in req_index:
    perturbed_text =' '.join([re.split(r'\W+',text)[i] for i,x in enumerate(data[k]) if x==1.0])
    closest_perturbed_sample.append(perturbed_text)
closest_perturbed_sample

In [ ]:
ret_exp.show_in_notebook(text=True)

In [ ]:
ret_exp.intercept

In [ ]:
from lime.lime_text import TextDomainMapper
from lime import lime_base
import numpy

def kernel(d, kernel_width):
    return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

kernel_fn = partial(kernel, kernel_width=25)

random_state=numpy.random.RandomState()

base = lime_base.LimeBase(kernel_fn, verbose=False,
                                       random_state=random_state)

domain_mapper = TextDomainMapper(MyIndexedString(text4, bow = False))
class_names = [str(x) for x in range(yss[0].shape[0])]
ret_exp = explanation.Explanation(domain_mapper=domain_mapper,
                                    class_names=class_names,
                                    random_state=random_state)
ret_exp.predict_proba = yss[0]

for label in (1,):
    (ret_exp.intercept[label],
    ret_exp.local_exp[label],
    ret_exp.score[label],
    ret_exp.local_pred[label]) = base.explain_instance_with_data(
        data, yss, distances, label, num_features=5,
        model_regressor=None,
        feature_selection='auto')


In [ ]:
b = base.explain_instance_with_data(
        data, yss, distances, label, num_features=5,
        model_regressor=None,
        feature_selection='auto')

In [ ]:
len(b)

In [ ]:
ret_exp.local_pred

In [ ]:
for label in (1,):
    (ret_exp.intercept[label],
    ret_exp.local_exp[label],
    ret_exp.score[label],
    ret_exp.local_pred[label]) = b

In [ ]:
ret_exp.score

In [ ]:
from functools import partial
import itertools
import json
import re

import numpy as np
import scipy as sp
import sklearn
from sklearn.utils import check_random_state

from lime import lime_base

from lime.lime_text import TextDomainMapper, LimeTextExplainer

class MyLimeTextExplainer(LimeTextExplainer):
    """Explains text classifiers.
       Currently, we are using an exponential kernel on cosine distance, and
       restricting explanations to words that are present in documents."""

    def __init__(self,
                 kernel_width=25,
                 kernel=None,
                 verbose=False,
                 class_names=None,
                 feature_selection='auto',
                 split_expression=r'\W+',
                 bow=True,
                 mask_string=None,
                 random_state=None,
                 char_level=False,
                 mode='rand'):
        """Init function.
        Args:
            kernel_width: kernel width for the exponential kernel.
            kernel: similarity kernel that takes euclidean distances and kernel
                width as input and outputs weights in (0,1). If None, defaults to
                an exponential kernel.
            verbose: if true, print local prediction values from linear model
            class_names: list of class names, ordered according to whatever the
                classifier is using. If not present, class names will be '0',
                '1', ...
            feature_selection: feature selection method. can be
                'forward_selection', 'lasso_path', 'none' or 'auto'.
                See function 'explain_instance_with_data' in lime_base.py for
                details on what each of the options does.
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True (bag of words), will perturb input data by removing
                all occurrences of individual words or characters.
                Explanations will be in terms of these words. Otherwise, will
                explain in terms of word-positions, so that a word may be
                important the first time it appears and unimportant the second.
                Only set to false if the classifier uses word order in some way
                (bigrams, etc), or if you set char_level=True.
            mask_string: String used to mask tokens or characters if bow=False
                if None, will be 'UNKWORDZ' if char_level=False, chr(0)
                otherwise.
            random_state: an integer or numpy.RandomState that will be used to
                generate random numbers. If None, the random state will be
                initialized using the internal numpy seed.
            char_level: an boolean identifying that we treat each character
                as an independent occurence in the string
        """

        if kernel is None:
            def kernel(d, kernel_width):
                return np.sqrt(np.exp(-(d ** 2) / kernel_width ** 2))

        kernel_fn = partial(kernel, kernel_width=kernel_width)

        self.random_state = check_random_state(random_state)
        self.base = lime_base.LimeBase(kernel_fn, verbose,
                                       random_state=self.random_state)
        self.class_names = class_names
        self.vocabulary = None
        self.feature_selection = feature_selection
        self.bow = bow
        self.mask_string = mask_string
        self.split_expression = split_expression
        self.char_level = char_level
        self.mode = mode

    def explain_instance(self,
                         text_instance,
                         classifier_fn,
                         labels=(1,),
                         top_labels=None,
                         num_features=10,
                         num_samples=5000,
                         distance_metric='cosine',
                         model_regressor=None):
        """Generates explanations for a prediction.
        First, we generate neighborhood data by randomly hiding features from
        the instance (see __data_labels_distance_mapping). We then learn
        locally weighted linear models on this neighborhood data to explain
        each of the classes in an interpretable way (see lime_base.py).
        Args:
            text_instance: raw text string to be explained.
            classifier_fn: classifier prediction probability function, which
                takes a list of d strings and outputs a (d, k) numpy array with
                prediction probabilities, where k is the number of classes.
                For ScikitClassifiers , this is classifier.predict_proba.
            labels: iterable with labels to be explained.
            top_labels: if not None, ignore labels and produce explanations for
                the K labels with highest prediction probabilities, where K is
                this parameter.
            num_features: maximum number of features present in explanation
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity
            model_regressor: sklearn regressor to use in explanation. Defaults
            to Ridge regression in LimeBase. Must have model_regressor.coef_
            and 'sample_weight' as a parameter to model_regressor.fit()
        Returns:
            An Explanation object (see explanation.py) with the corresponding
            explanations.
        """

        indexed_string = (IndexedCharacters(
            text_instance, bow=self.bow, mask_string=self.mask_string)
                          if self.char_level else
                          MyIndexedString(text_instance, bow=self.bow,
                                        split_expression=self.split_expression,
                                        mask_string=self.mask_string, mode=self.mode))
        domain_mapper = TextDomainMapper(indexed_string)
        data, yss, distances = self.__data_labels_distances(
            indexed_string, classifier_fn, num_samples,
            distance_metric=distance_metric, mode = self.mode)
        if self.class_names is None:
            self.class_names = [str(x) for x in range(yss[0].shape[0])]
        #ret_exp = explanation.Explanation(domain_mapper=domain_mapper,
        ret_exp = MyExplanation(domain_mapper=domain_mapper,
                                          class_names=self.class_names,
                                          random_state=self.random_state)
        ret_exp.predict_proba = yss[0]
        if top_labels:
            labels = np.argsort(yss[0])[-top_labels:]
            ret_exp.top_labels = list(labels)
            ret_exp.top_labels.reverse()
        for label in labels:
            (ret_exp.intercept[label],
             ret_exp.local_exp[label],
             ret_exp.score[label],
             ret_exp.local_pred[label]) = self.base.explain_instance_with_data(
                data, yss, distances, label, num_features,
                model_regressor=model_regressor,
                feature_selection=self.feature_selection)
        return ret_exp

    def __data_labels_distances(self,
                                indexed_string,
                                classifier_fn,
                                num_samples,
                                distance_metric='cosine',
                                mode='rand'):
        """Generates a neighborhood around a prediction.
        Generates neighborhood data by randomly removing words from
        the instance, and predicting with the classifier. Uses cosine distance
        to compute distances between original and perturbed instances.
        Args:
            indexed_string: document (IndexedString) to be explained,
            classifier_fn: classifier prediction probability function, which
                takes a string and outputs prediction probabilities. For
                ScikitClassifier, this is classifier.predict_proba.
            num_samples: size of the neighborhood to learn the linear model
            distance_metric: the distance metric to use for sample weighting,
                defaults to cosine similarity.
        Returns:
            A tuple (data, labels, distances), where:
                data: dense num_samples * K binary matrix, where K is the
                    number of tokens in indexed_string. The first row is the
                    original instance, and thus a row of ones.
                labels: num_samples * L matrix, where L is the number of target
                    labels
                distances: cosine distance between the original instance and
                    each perturbed instance (computed in the binary 'data'
                    matrix), times 100.
        """
        def distance_fn(x):
            return sklearn.metrics.pairwise.pairwise_distances(
                    x, x[0], metric=distance_metric).ravel() * 100

        doc_size = indexed_string.num_words()

        sep = indexed_string.return_sep()
        print("sep: ", sep, "docsize: ", doc_size)

        global sample
        sample = self.random_state.randint(1, doc_size + 1, num_samples - 1)

        data = np.ones((num_samples, doc_size))
        data[0] = np.ones(doc_size)
        inverse_data = [indexed_string.raw_string()]

        if not indexed_string.bow: 
            
            if sep < doc_size:
                print("separation on")

                sample_left = self.random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = self.random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(sep, doc_size)

                if mode == 'left':
                    print("left")
                    sample = sample_left
                    features_range = features_range_left

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                elif mode == 'right':
                    print("right")
                    sample = sample_right
                    features_range = features_range_right

                    #global save_sample
                    #save_sample = sample

                    for i, size in enumerate(sample, start=1):
                        print(i, len(features_range), size)
                        inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

                else:
                    print('rand')
                    dir = self.random_state.randint(2, size = num_samples - 1)

                    for i, size in enumerate(sample, start=1):
                        #print(dir[i-1])
                        if dir[i-1]:
                            inactive = self.random_state.choice(features_range_right, sample_right[i-1],
                                                        replace=False)
                        else:
                            inactive = self.random_state.choice(features_range_left, sample_left[i-1],
                                                        replace=False)
                        data[i, inactive] = 0
                        inverse_data.append(indexed_string.inverse_removing(inactive))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))

        else:

            #added in lime22
            if indexed_string.positions_right:
                print("separation on, bow mode")

                sample_left = self.random_state.randint(1, sep + 1, num_samples - 1)
                sample_right = self.random_state.randint(1, doc_size - sep, num_samples - 1)

                features_range_left = range(0, sep)
                features_range_right = range(0, doc_size - sep)

                #global save_sample
                #save_sample = sample

                if mode == 'left':
                    print("left bow")
                    sample = sample_left
                    #print("sample: ", sample)
                    features_range = features_range_left

                elif mode == 'right':
                    print("right bow")
                    sample = sample_right
                    #print("sample: ", sample)
                    features_range = features_range_right

                for i, size in enumerate(sample, start=1):
                    #print(i, len(features_range), size)
                    inactive = self.random_state.choice(features_range, size,
                                                    replace=False)
                    data[i, inactive] = 0
                    #print('inactive', inactive)
                    inverse_data.append(indexed_string.inverse_removing_modes(inactive, mode=mode))

            else:
                features_range = range(doc_size)
                for i, size in enumerate(sample, start=1):
                    inactive = self.random_state.choice(features_range, size,
                                                        replace=False)
                    data[i, inactive] = 0
                    inverse_data.append(indexed_string.inverse_removing(inactive))


        labels = classifier_fn(inverse_data)
        distances = distance_fn(sp.sparse.csr_matrix(data))
        return data, labels, distances

In [ ]:
class IndexedCharacters(object):
    """String with various indexes."""

    def __init__(self, raw_string, bow=True, mask_string=None):
        """Initializer.
        Args:
            raw_string: string with raw text in it
            bow: if True, a char is the same everywhere in the text - i.e. we
                 will index multiple occurrences of the same character. If False,
                 order matters, so that the same word will have different ids
                 according to position.
            mask_string: If not None, replace characters with this if bow=False
                if None, default value is chr(0)
        """
        self.raw = raw_string
        self.as_list = list(self.raw)
        self.as_np = np.array(self.as_list)
        self.mask_string = chr(0) if mask_string is None else mask_string
        self.string_start = np.arange(len(self.raw))
        vocab = {}
        self.inverse_vocab = []
        self.positions = []
        self.bow = bow
        non_vocab = set('$','&','*')
        for i, char in enumerate(self.as_np):
            if char in non_vocab:
                continue
            if bow:
                if char not in vocab:
                    vocab[char] = len(vocab)
                    self.inverse_vocab.append(char)
                    self.positions.append([])
                idx_char = vocab[char]
                self.positions[idx_char].append(i)
            else:
                self.inverse_vocab.append(char)
                self.positions.append(i)
        if not bow:
            self.positions = np.array(self.positions)

    def raw_string(self):
        """Returns the original raw string"""
        return self.raw

    def num_words(self):
        """Returns the number of tokens in the vocabulary for this document."""
        return len(self.inverse_vocab)

    def word(self, id_):
        """Returns the word that corresponds to id_ (int)"""
        return self.inverse_vocab[id_]

    def string_position(self, id_):
        """Returns a np array with indices to id_ (int) occurrences"""
        if self.bow:
            return self.string_start[self.positions[id_]]
        else:
            return self.string_start[[self.positions[id_]]]

    def inverse_removing(self, words_to_remove):
        """Returns a string after removing the appropriate words.
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        mask[self.__get_idxs(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    def __get_idxs(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions[z] for z in words]))
        else:
            return self.positions[words]

In [ ]:
class MyIndexedString(object):
    """String with various indexes."""

    def __init__(self, raw_string, split_expression=r'\W+', bow=True,
                 mask_string=None, mode='left'):
        """Initializer.
        Args:
            raw_string: string with raw text in it
            split_expression: Regex string or callable. If regex string, will be used with re.split.
                If callable, the function should return a list of tokens.
            bow: if True, a word is the same everywhere in the text - i.e. we
                 will index multiple occurrences of the same word. If False,
                 order matters, so that the same word will have different ids
                 according to position.
            mask_string: If not None, replace words with this if bow=False
                if None, default value is UNKWORDZ
        """
        self.raw = raw_string
        self.mask_string = 'UNKWORDZ' if mask_string is None else mask_string

        if callable(split_expression):
            tokens = split_expression(self.raw)
            self.as_list = self._segment_with_tokens(self.raw, tokens)
            tokens = set(tokens)

            def non_word(string):
                return string not in tokens

        else:
            # with the split_expression as a non-capturing group (?:), we don't need to filter out
            # the separator character from the split results.
            splitter = re.compile(r'(%s)|$' % split_expression)
            self.as_list = [s for s in splitter.split(self.raw) if s]
            non_word = splitter.match

        self.as_np = np.array(self.as_list)
        self.string_start = np.hstack(
            ([0], np.cumsum([len(x) for x in self.as_np[:-1]])))
        vocab = {}
        self.inverse_vocab = []
        self.positions = []
        self.bow = bow

        #added in lime22
        self.sep = None
        self.mode = mode

        #added in lime22
        vocab_right = {}
        self.inverse_vocab_right = []
        self.positions_right = []

        non_vocab = set()
        for i, word in enumerate(self.as_np):
            if "$&*&*&$" in word:
                self.sep = len(self.inverse_vocab)
            if word in non_vocab:
                continue
            if non_word(word):
                non_vocab.add(word)
                continue
            if bow:
                #added in lime22
                if not self.sep:
                    if word not in vocab:
                        vocab[word] = len(vocab)
                        self.inverse_vocab.append(word)
                        self.positions.append([])
                    idx_word = vocab[word]
                    #print(1, idx_word)
                    self.positions[idx_word].append(i)
                else:
                    if word not in vocab_right:
                        vocab_right[word] = len(vocab_right)
                        self.inverse_vocab_right.append(word)
                        self.positions_right.append([])
                    idx_word = vocab_right[word]
                    #print(2, idx_word)
                    self.positions_right[idx_word].append(i)
                #was originally
                """if word not in vocab:
                    vocab[word] = len(vocab)
                    self.inverse_vocab.append(word)
                    self.positions.append([])
                idx_word = vocab[word]
                self.positions[idx_word].append(i)"""
            else:
                self.inverse_vocab.append(word)
                self.positions.append(i)
        if not bow:
            self.positions = np.array(self.positions)

        if not self.sep:
            self.sep = len(self.inverse_vocab)

    def return_sep(self):
        """Return the index of the separator sequence"""
        return self.sep

    def raw_string(self):
        """Returns the original raw string"""
        return self.raw

    def num_words(self):
        """Returns the number of tokens in the vocabulary for this document."""
        #added in lime22
        return len(self.inverse_vocab) if not self.inverse_vocab_right else len(self.inverse_vocab) + len(self.inverse_vocab_right)

    def word(self, id_):
        #print(self.mode)
        """Returns the word that corresponds to id_ (int)"""
        if not self.inverse_vocab_right:
            return self.inverse_vocab[id_]
        #added in lime22
        else:
            if self.mode == 'left':
                #print('left', self.inverse_vocab[id_])
                return self.inverse_vocab[id_]
            if self.mode == 'right':
                #print('right', self.inverse_vocab_right[id_])
                return self.inverse_vocab_right[id_]
            else:
                raise ValueError("BOW only supports 'left' and 'right' modes")


    def string_position(self, id_):
        """Returns a np array with indices to id_ (int) occurrences"""
        if self.bow:
            #added in lime22
            if self.mode == 'left':
                #print(self.string_start[self.positions[id_]])
                return self.string_start[self.positions[id_]]
            if self.mode == 'right':
                #print(self.string_start[self.positions_right[id_]])
                return self.string_start[self.positions_right[id_]]
        else:
            return self.string_start[[self.positions[id_]]]

    def inverse_removing(self, words_to_remove):
        """Returns a string after removing the appropriate words.
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        mask[self.__get_idxs(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    #added in lime22
    def inverse_removing_modes(self, words_to_remove, mode='left'):
        """Returns a string after removing the appropriate words.
        Removes the anappropriate words from the text BEFIORE ('left') or AFTER
        ('right') the separator
        If self.bow is false, replaces word with UNKWORDZ instead of removing
        it.
        Args:
            words_to_remove: list of ids (ints) to remove
        Returns:
            original raw string with appropriate words removed.
        """
        mask = np.ones(self.as_np.shape[0], dtype='bool')
        if mode == "left":
            mask[self.__get_idxs(words_to_remove)] = False
        elif mode == "right":
            mask[self.__get_idxs_right(words_to_remove)] = False
        if not self.bow:
            return ''.join(
                [self.as_list[i] if mask[i] else self.mask_string
                 for i in range(mask.shape[0])])
        return ''.join([self.as_list[v] for v in mask.nonzero()[0]])

    @staticmethod
    def _segment_with_tokens(text, tokens):
        """Segment a string around the tokens created by a passed-in tokenizer"""
        list_form = []
        text_ptr = 0
        for token in tokens:
            inter_token_string = []
            while not text[text_ptr:].startswith(token):
                inter_token_string.append(text[text_ptr])
                text_ptr += 1
                if text_ptr >= len(text):
                    raise ValueError("Tokenization produced tokens that do not belong in string!")
            text_ptr += len(token)
            if inter_token_string:
                list_form.append(''.join(inter_token_string))
            list_form.append(token)
        if text_ptr < len(text):
            list_form.append(text[text_ptr:])
        return list_form

    def __get_idxs(self, words):
        """Returns indexes to appropriate words."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions[z] for z in words]))
        else:
            return self.positions[words]

    #added in lime22
    def __get_idxs_right(self, words):
        """Returns indexes to appropriate words for the text after the separator."""
        if self.bow:
            return list(itertools.chain.from_iterable(
                [self.positions_right[z] for z in words]))
        else:
            return self.positions_right[words]

In [ ]:
from io import open
import os
import os.path
import json
import string
import numpy as np

from lime.exceptions import LimeError
import lime.explanation

from sklearn.utils import check_random_state

def id_generator(size=15, random_state=None):
    """Helper function to generate random div ids. This is useful for embedding
    HTML into ipython notebooks."""
    chars = list(string.ascii_uppercase + string.digits)
    return ''.join(random_state.choice(chars, size, replace=True))


class DomainMapper(object):
    """Class for mapping features to the specific domain.
    The idea is that there would be a subclass for each domain (text, tables,
    images, etc), so that we can have a general Explanation class, and separate
    out the specifics of visualizing features in here.
    """

    def __init__(self):
        pass

    def map_exp_ids(self, exp, **kwargs):
        """Maps the feature ids to concrete names.
        Default behaviour is the identity function. Subclasses can implement
        this as they see fit.
        Args:
            exp: list of tuples [(id, weight), (id,weight)]
            kwargs: optional keyword arguments
        Returns:
            exp: list of tuples [(name, weight), (name, weight)...]
        """
        return exp

    def visualize_instance_html(self,
                                exp,
                                label,
                                div_name,
                                exp_object_name,
                                **kwargs):
        """Produces html for visualizing the instance.
        Default behaviour does nothing. Subclasses can implement this as they
        see fit.
        Args:
             exp: list of tuples [(id, weight), (id,weight)]
             label: label id (integer)
             div_name: name of div object to be used for rendering(in js)
             exp_object_name: name of js explanation object
             kwargs: optional keyword arguments
        Returns:
             js code for visualizing the instance
        """
        return ''


class MyExplanation(object):
    """Object returned by explainers."""

    def __init__(self,
                 domain_mapper,
                 mode='classification',
                 class_names=None,
                 random_state=None):
        """
        Initializer.
        Args:
            domain_mapper: must inherit from DomainMapper class
            type: "classification" or "regression"
            class_names: list of class names (only used for classification)
            random_state: an integer or numpy.RandomState that will be used to
                generate random numbers. If None, the random state will be
                initialized using the internal numpy seed.
        """
        self.random_state = random_state
        self.mode = mode
        self.domain_mapper = domain_mapper
        self.local_exp = {}
        self.intercept = {}
        self.score = {}
        self.local_pred = {}
        if mode == 'classification':
            self.class_names = class_names
            self.top_labels = None
            self.predict_proba = None
        elif mode == 'regression':
            self.class_names = ['negative', 'positive']
            self.predicted_value = None
            self.min_value = 0.0
            self.max_value = 1.0
            self.dummy_label = 1
        else:
            raise LimeError('Invalid explanation mode "{}". '
                            'Should be either "classification" '
                            'or "regression".'.format(mode))

    def available_labels(self):
        """
        Returns the list of classification labels for which we have any explanations.
        """
        try:
            assert self.mode == "classification"
        except AssertionError:
            raise NotImplementedError('Not supported for regression explanations.')
        else:
            ans = self.top_labels if self.top_labels else self.local_exp.keys()
            return list(ans)

    def as_list(self, label=1, **kwargs):
        """Returns the explanation as a list.
        Args:
            label: desired label. If you ask for a label for which an
                explanation wasn't computed, will throw an exception.
                Will be ignored for regression explanations.
            kwargs: keyword arguments, passed to domain_mapper
        Returns:
            list of tuples (representation, weight), where representation is
            given by domain_mapper. Weight is a float.
        """
        label_to_use = label if self.mode == "classification" else self.dummy_label
        ans = self.domain_mapper.map_exp_ids(self.local_exp[label_to_use], **kwargs)
        ans = [(x[0], float(x[1])) for x in ans]
        return ans

    def as_map(self):
        """Returns the map of explanations.
        Returns:
            Map from label to list of tuples (feature_id, weight).
        """
        return self.local_exp

    def as_pyplot_figure(self, label=1, figsize=(4,4), **kwargs):
        """Returns the explanation as a pyplot figure.
        Will throw an error if you don't have matplotlib installed
        Args:
            label: desired label. If you ask for a label for which an
                   explanation wasn't computed, will throw an exception.
                   Will be ignored for regression explanations.
            figsize: desired size of pyplot in tuple format, defaults to (4,4).
            kwargs: keyword arguments, passed to domain_mapper
        Returns:
            pyplot figure (barchart).
        """
        import matplotlib.pyplot as plt
        exp = self.as_list(label=label, **kwargs)
        fig = plt.figure(figsize=figsize)
        vals = [x[1] for x in exp]
        names = [x[0] for x in exp]
        vals.reverse()
        names.reverse()
        colors = ['green' if x > 0 else 'red' for x in vals]
        pos = np.arange(len(exp)) + .5
        plt.barh(pos, vals, align='center', color=colors)
        plt.yticks(pos, names)
        if self.mode == "classification":
            title = 'Local explanation for class %s' % self.class_names[label]
        else:
            title = 'Local explanation'
        plt.title(title)
        return fig

    def show_in_notebook(self,
                         labels=None,
                         predict_proba=True,
                         show_predicted_value=True,
                         **kwargs):
        """Shows html explanation in ipython notebook.
        See as_html() for parameters.
        This will throw an error if you don't have IPython installed"""

        from IPython.core.display import display, HTML
        display(HTML(self.as_html(labels=labels,
                                  predict_proba=predict_proba,
                                  show_predicted_value=show_predicted_value,
                                  **kwargs)))

    def save_to_file(self,
                     file_path,
                     labels=None,
                     predict_proba=True,
                     show_predicted_value=True,
                     **kwargs):
        """Saves html explanation to file. .
        Params:
            file_path: file to save explanations to
        See as_html() for additional parameters.
        """
        file_ = open(file_path, 'w', encoding='utf8')
        file_.write(self.as_html(labels=labels,
                                 predict_proba=predict_proba,
                                 show_predicted_value=show_predicted_value,
                                 **kwargs))
        file_.close()

    def as_html(self,
                labels=None,
                predict_proba=True,
                show_predicted_value=True,
                **kwargs):
        """Returns the explanation as an html page.
        Args:
            labels: desired labels to show explanations for (as barcharts).
                If you ask for a label for which an explanation wasn't
                computed, will throw an exception. If None, will show
                explanations for all available labels. (only used for classification)
            predict_proba: if true, add  barchart with prediction probabilities
                for the top classes. (only used for classification)
            show_predicted_value: if true, add  barchart with expected value
                (only used for regression)
            kwargs: keyword arguments, passed to domain_mapper
        Returns:
            code for an html page, including javascript includes.
        """

        def jsonize(x):
            return json.dumps(x, ensure_ascii=False)

        if labels is None and self.mode == "classification":
            labels = self.available_labels()

        this_dir, _ = os.path.split(lime.explanation.__file__)
        bundle = open(os.path.join(this_dir, 'bundle.js'),
                      encoding="utf8").read()

        out = u'''<html>
        <meta http-equiv="content-type" content="text/html; charset=UTF8">
        <head><script>%s </script></head><body>''' % bundle
        random_id = id_generator(size=15, random_state=check_random_state(self.random_state))
        out += u'''
        <div class="lime top_div" id="top_div%s"></div>
        ''' % random_id

        predict_proba_js = ''
        if self.mode == "classification" and predict_proba:
            predict_proba_js = u'''
            var pp_div = top_div.append('div')
                                .classed('lime predict_proba', true);
            var pp_svg = pp_div.append('svg').style('width', '100%%');
            var pp = new lime.PredictProba(pp_svg, %s, %s);
            ''' % (jsonize([str(x) for x in self.class_names]),
                   jsonize(list(self.predict_proba.astype(float))))

        predict_value_js = ''
        if self.mode == "regression" and show_predicted_value:
            # reference self.predicted_value
            # (svg, predicted_value, min_value, max_value)
            predict_value_js = u'''
                    var pp_div = top_div.append('div')
                                        .classed('lime predicted_value', true);
                    var pp_svg = pp_div.append('svg').style('width', '100%%');
                    var pp = new lime.PredictedValue(pp_svg, %s, %s, %s);
                    ''' % (jsonize(float(self.predicted_value)),
                           jsonize(float(self.min_value)),
                           jsonize(float(self.max_value)))

        exp_js = '''var exp_div;
            var exp = new lime.Explanation(%s);
        ''' % (jsonize([str(x) for x in self.class_names]))

        if self.mode == "classification":
            for label in labels:
                exp = jsonize(self.as_list(label))
                exp_js += u'''
                exp_div = top_div.append('div').classed('lime explanation', true);
                exp.show(%s, %d, exp_div);
                ''' % (exp, label)
        else:
            exp = jsonize(self.as_list())
            exp_js += u'''
            exp_div = top_div.append('div').classed('lime explanation', true);
            exp.show(%s, %s, exp_div);
            ''' % (exp, self.dummy_label)

        raw_js = '''var raw_div = top_div.append('div');'''

        if self.mode == "classification":
            html_data = self.local_exp[labels[0]]
        else:
            html_data = self.local_exp[self.dummy_label]

        raw_js += self.domain_mapper.visualize_instance_html(
                html_data,
                labels[0] if self.mode == "classification" else self.dummy_label,
                'raw_div',
                'exp',
                **kwargs)
        out += u'''
        <script>
        var top_div = d3.select('#top_div%s').classed('lime top_div', true);
        %s
        %s
        %s
        %s
        </script>
        ''' % (random_id, predict_proba_js, predict_value_js, exp_js, raw_js)
        out += u'</body></html>'

        return out